In [1]:
# Dependencies
import time
from splinter import Browser
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait  # Import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
# Use the read_csv function to read the CSV file
df = pd.read_csv("Resources/02_top_ten_asx_by_industry.csv")

df.head(10)

,ticker,company_name,industry_gp,market_cap
0,ARB,ARB CORPORATION LIMITED.,Automobiles & Components,2.455925e+09
1,GUD,G.U.D. HOLDINGS LIMITED,Automobiles & Components,1.534343e+09
2,PWH,PWR HOLDINGS LIMITED,Automobiles & Components,1.000822e+09
3,SFC,SCHAFFER CORPORATION LIMITED,Automobiles & Components,2.272901e+08
4,AHL,ADRAD HOLDINGS LIMITED,Automobiles & Components,7.275023e+07
5,VMT,VMOTO LIMITED,Automobiles & Components,4.353604e+07
6,RPM,RPM AUTOMOTIVE GROUP LIMITED,Automobiles & Components,1.573357e+07
7,ABV,ADVANCED BRAKING TECHNOLOGY LTD,Automobiles & Components,1.555814e+07
8,DDT,DATADOT TECHNOLOGY LIMITED,Automobiles & Components,4.843811e+06
9,SIX,SPRINTEX LIMITED,Automobiles & Components,4.745972e+06


In [3]:
# Extract the "Code" column as a list
all_tickers = df['ticker'].tolist()

In [4]:
print(all_tickers)

['ARB', 'GUD', 'PWH', 'SFC', 'AHL', 'VMT', 'RPM', 'ABV', 'DDT', 'SIX', 'CBA', 'NAB', 'ANZ', 'WBC', 'BEN', 'VUK', 'BOQ', 'BFL', 'HGH', 'JDO', 'REH', 'SVW', 'WOR', 'IFT', 'FBU', 'RWC', 'VNT', 'JLG', 'MND', 'RDX', 'SLB', 'TRE', 'OXT', 'S3N', 'HRE', 'KLI', 'DY6', 'BXB', 'CPU', 'ALQ', 'CWY', 'DOW', 'APM', 'IPH', 'MAD', 'MMS', 'SIQ', 'WES', 'JBH', 'HVN', 'PMV', 'APE', 'SUL', 'LOV', 'BAP', 'AX1', 'BGP', 'BRG', 'AKP', 'FWD', 'WAT', 'AVJ', 'GLB', 'TWD', 'SHM', 'SIO', 'GAP', 'ALL', 'LNW', 'TLC', 'IEL', 'DMP', 'FLT', 'CTD', 'WEB', 'TAH', 'IVC', 'WOW', 'COL', 'EDV', 'MTS', 'GNC', 'MMM', 'WDS', 'STO', 'ALD', 'YAL', 'WHC', 'NXG', 'NHC', 'VEA', 'BPT', 'PDN', 'GMG', 'SCG', 'URW', 'SGP', 'VCX', 'MGR', 'GPT', 'DXS', 'CHC', 'NSR', 'MQG', 'SQ2', 'SOL', 'ASX', 'JHG', 'CGF', 'GQG', 'NWL', 'AMP', 'HUB', 'TWE', 'A2M', 'CGC', 'ING', 'ELD', 'BGA', 'AAC', 'RIC', 'CBO', 'SHV', 'RMD', 'COH', 'SHL', 'FPH', 'RHC', 'PME', 'EBO', 'ANN', 'SNZ', 'NAN', 'PTL', 'MCP', 'BIO', 'BXN', 'S66', 'HPC', 'EXL', 'SKN', 'CCO', 'HCT'

## Define function to scape the data from website using the ticker

In [8]:
def scrape_stock_data(ticker):
    # Set up Splinter
    browser = Browser('chrome')

    # Define the URL of the website you want to scrape for the given ticker
    url = f"https://www.marketindex.com.au/asx/{ticker}"

    # Use Splinter to open the URL
    browser.visit(url)
    
    #  Wait for website to load
    # website has anti bot protection.  Sometimes, the scarping doesn't work all at once.
    time.sleep(5)
    
    # Wait for a specific div element with a certain class to appear
    wait = WebDriverWait(browser.driver, 10)
    element_to_wait_for = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.sm\\:w-1\\/2.lg\\:w-full.sm\\:pr-4.lg\\:p-0')))
    
    # Give another 5 seconds wait
    time.sleep(5)
    
    # Get the HTML content from the page
    html_content = browser.html
    
    # Create a BeautifulSoup object to parse the HTML
    soup = BeautifulSoup(html_content, 'html.parser')

    # Define a function to extract data from a table
    def extract_data_from_table(table):
        data = {}
        rows = table.find_all('tr')
        for row in rows:
            columns = row.find_all('td')
            if len(columns) == 2:
                attribute = columns[0].text.strip()
                value = columns[1].text.strip()
                data[attribute] = value
        return data

    # Find and extract data from different tables
    # Find and extract data from the "Share Price Activity (ASX)" table
    share_price_table = soup.find('h3', string='Share Price Activity (ASX)')
    if share_price_table:
        share_price_table = share_price_table.find_next('table', class_='mi-table')
        share_price_data = extract_data_from_table(share_price_table)
    else:
        share_price_data = {}  # or another default value or error handling

    # Find and extract data from the "Performance)" table
    performance_table = soup.find('h3', string='Performance')
    if performance_table:
        performance_table = performance_table.find_next('table', class_='mi-table')
        performance_data = extract_data_from_table(performance_table)
    else:
        performance_data = {}  # or another default value or error handling

    # Find and extract data from the "Size" table
    size_table = soup.find('h3', string='Size')
    if size_table:
        size_table = size_table.find_next('table', class_='mi-table')
        size_data = extract_data_from_table(size_table)
    else:
        size_data = {}  # or another default value or error handling

    # Find and extract data from the "Key Fundamentals" table
    fundamentals_table = soup.find('h3', string='Key Fundamentals')
    if fundamentals_table:
        fundamentals_table = fundamentals_table.find_next('table', class_='mi-table')
        fundamentals_data = extract_data_from_table(fundamentals_table)
    else:
        fundamentals_data = {}  # or another default value or error handling

    # Find and extract data from the "Broker Consensus" table
    broker_consensus_table = soup.find('h3', string='Broker Consensus')
    if broker_consensus_table:
        broker_consensus_table = broker_consensus_table.find_next('table', class_='mi-table')
        broker_consensus_data = extract_data_from_table(broker_consensus_table)
    else:
        broker_consensus_data = {}  # or another default value or error handling

    # Combine all the extracted data into one dictionary
    all_data = {**share_price_data, **performance_data, **size_data, **fundamentals_data, **broker_consensus_data}

    # Create a DataFrame
    df = pd.DataFrame(list(all_data.items()), columns=['Attribute', 'Value'])

    # Transpose the DataFrame
    df_transposed = df.T

    # Set the "Attribute" column as the index
    df_transposed.columns = df_transposed.loc['Attribute']

    # Remove the duplicated "Attribute" row
    df_transposed = df_transposed[1:]

    # Reset the index and add a "Ticker" column with the given ticker
    df_transposed = df_transposed.reset_index()
    df_transposed = df_transposed.rename(columns={'Attribute': 'Ticker'})
    df_transposed['Ticker'] = ticker

    # Close the browser
    browser.quit()

    return df_transposed

# Example usage:
ticker = 'NAB'  # Replace with the desired ticker
nab_df = scrape_stock_data(ticker)
nab_df

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$29.06,0.49\n(1.72%),$28.92\n-\n$29.10,"4,699,140","4,334,055","$136,415,820",$28.95,$28.78 - $29.13,$29.025,...,"3,128,948,927",Financial Services,ANZ / WBC / MQG,$2.268,$1.61,$17.372,9 Buy\n· 5 Hold\n· 4 Sell,Hold,01/10/23,NAB


In [10]:
# Please note that scarping the website for 259 tickers took mroe than 1 hour and some of the ticker
# were not scraped successfully.  Those have to be extracted individualy and added to the final dataframe.

# To mitigate this, the scarping was done about 10 tickers at a time and later append the dataframe
# using the function below.

def scrape_multiple_stock_data(tickers):
    # Initialize an empty list to store DataFrames
    dfs = []

    # Iterate through tickers
    for ticker in tickers:
        df = scrape_stock_data(ticker)
        dfs.append(df)

    # Concatenate the list of DataFrames into a single DataFrame
    final_df = pd.concat(dfs, ignore_index=True)

    return final_df

In [11]:
# Output 1st 10 tickers
tickers = all_tickers[0:10]  # Replace with the desired tickers
final_df1 = scrape_multiple_stock_data(tickers)
final_df1

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$31.26,1.02\n(3.37%),$29.88\n-\n$31.35,"160,245","245,483","$4,999,714",$31.07,$30.56 - $31.45,$31.20,...,"82,220,441",Consumer Cyclical,CTD / WEB / SUL,$1.079,$0.62,$0.60,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,ARB
1,Value,$11.08,0.17\n(1.56%),$10.77\n-\n$11.35,"366,353","441,797","$4,045,735",$10.95,$10.725 - $11.10,$11.045,...,"140,894,696",Consumer Cyclical,SGR / LNW / SKC,$0.693,$0.39,$0.526,11 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,GUD
2,Value,$10.12,0.07\n(0.70%),$9.96\n-\n$10.92,"37,398","127,350","$378,557",$10.16,$10.06 - $10.22,$10.12,...,"100,484,131",Consumer Cyclical,BGP / CTT / AX1,$0.217,$0.125,$0.722,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,PWH
3,Value,$16.10,0.65\n(-3.88%),$16.15\n-\n$17.10,"5,570",758,"$90,352",$16.75,$16.01 - $16.75,$16.22,...,"13,569,557",Consumer Cyclical,NDO / ALG / BST,$1.811,$0.90,$3.324,NaN,NaN,NaN,SFC
4,Value,$0.89,0.00\n(0.00),$0.845\n-\n$0.90,0,"11,146",$0,‐,‐ - ‐,‐,...,"81,285,172",Consumer Cyclical,ISU / KED / CCX,$0.084,$0.0233,$0.986,NaN,NaN,NaN,AHL
5,Value,$0.155,0.005\n(3.33%),$0.145\n-\n$0.155,"82,262","432,421","$12,513",$0.155,$0.145 - $0.155,$0.152,...,"290,240,290",Consumer Cyclical,DNA / BBT / MFD,$0.025,$0.00,$0.202,NaN,NaN,NaN,VMT
6,Value,$0.081,0.0005\n(0.62%),$0.078\n-\n$0.085,"92,000","26,007","$7,490",$0.081,$0.081 - $0.082,$0.0815,...,"195,448,061",Consumer Cyclical,XRG / ABV / CLU,$0.01,$0.00,$0.031,NaN,NaN,NaN,RPM
7,Value,$0.044,0.002\n(4.76%),$0.043\n-\n$0.046,"231,792","154,298","$10,160",$0.042,$0.042 - $0.045,$0.044,...,"379,466,944",Consumer Cyclical,XRG / RPM / UCW,$0.004,$0.00,$0.017,NaN,NaN,NaN,ABV
8,Value,$0.004,0.00\n(0.00),$0.004\n-\n$0.005,"286,645","1,241,239","$1,147",$0.004,$0.004 - $0.004,$0.004,...,"1,210,952,783",Consumer Cyclical,MHI / SIX / AUA,$0.00,$0.00,$0.008,NaN,NaN,NaN,DDT
9,Value,$0.014,0.00\n(0.00),$0.012\n-\n$0.014,176,"555,355",$2,$0.014,$0.014 - $0.014,$0.014,...,"338,998,012",Consumer Cyclical,AUA / MHI / DDT,-$0.02,$0.00,$0.003,NaN,NaN,NaN,SIX


In [15]:
# Output 2nd 10 tickers
tickers = all_tickers[10:20]  # Replace with the desired tickers
final_df2 = scrape_multiple_stock_data(tickers)
final_df2

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$99.93,1.50\n(1.52%),$99.75\n-\n$99.95,"2,507,791","1,993,484","$250,439,945",$99.60,$99.20 - $99.99,$99.845,...,"1,675,527,631",Financial Services,NAB / ANZ / WBC,$5.893,$4.50,$38.582,0 Buy\n· 5 Hold\n· 15 Sell,Strong Sell,01/10/23,CBA
1,Value,$29.06,0.49\n(1.72%),$28.92\n-\n$29.10,"4,699,140","4,334,055","$136,415,820",$28.95,$28.78 - $29.13,$29.025,...,"3,128,948,927",Financial Services,ANZ / WBC / MQG,$2.268,$1.61,$17.372,9 Buy\n· 5 Hold\n· 4 Sell,Hold,01/10/23,NAB
2,Value,$25.60,0.23\n(0.91%),$25.59\n-\n$25.63,"5,813,929","5,496,181","$148,785,898",$25.73,$25.56 - $25.75,$25.635,...,"3,005,286,886",Financial Services,WBC / MQG / NAB,$2.297,$1.55,$20.682,12 Buy\n· 4 Hold\n· 2 Sell,Buy,01/10/23,ANZ
3,Value,$21.50,0.33\n(1.56%),$21.44\n-\n$21.50,"7,177,724","6,695,917","$154,169,117",$21.46,$21.39 - $21.58,$21.485,...,"3,509,076,960",Financial Services,ANZ / MQG / NAB,$1.744,$1.34,$17.161,3 Buy\n· 6 Hold\n· 10 Sell,Sell,01/10/23,WBC
4,Value,$9.09,0.20\n(2.25%),$9.05\n-\n$9.10,"1,285,689","1,174,747","$11,645,116",$8.96,$8.93 - $9.10,$9.055,...,"567,760,966",Financial Services,SDF / STW / MLT,$0.792,$0.61,$8.851,7 Buy\n· 8 Hold\n· 2 Sell,Hold,01/10/23,BEN
5,Value,$3.01,0.13\n(4.51%),$2.98\n-\n$3.02,"10,515,183","1,510,611","$31,555,235",$2.96,$2.94 - $3.015,$2.985,...,"672,361,526",Financial Services,IAF / VDHG / PDL,$0.337,$0.1922,$4.533,13 Buy\n· 8 Hold\n· 3 Sell,Buy,01/10/23,VUK
6,Value,$5.45,0.19\n(3.61%),$5.42\n-\n$5.45,"2,691,645","3,507,998","$14,564,520",$5.30,$5.30 - $5.45,$5.41,...,"657,217,431",Financial Services,NHF / VTS / NDQ,$0.183,$0.44,$7.734,2 Buy\n· 6 Hold\n· 12 Sell,Sell,01/10/23,BOQ
7,Value,$5.31,0.21\n(4.12%),$5.10\n-\n$5.35,"22,533","7,803","$116,762",$5.10,$5.10 - $5.35,$5.18,...,"467,219,979",Financial Services,VGAD / MGF / ETHI,$2.38,$0.6258,$7.958,NaN,NaN,NaN,BFL
8,Value,$1.415,0.00\n(0.00),$1.385\n-\n$1.63,0,928,$0,‐,‐ - ‐,‐,...,"715,265,270",Financial Services,VGB / DUI / JDO,$0.14,$0.1063,$1.121,6 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,HGH
9,Value,$0.885,0.005\n(0.57%),$0.88\n-\n$0.90,"981,281","1,347,745","$871,677",$0.88,$0.88 - $0.90,$0.8885,...,"1,109,300,833",Financial Services,GMA / VGB / HGH,$0.064,$0.00,$1.297,5 Buy\n· 7 Hold\n· 0 Sell,Buy,01/10/23,JDO


In [16]:
# Output 3rd 10 tickers
tickers = all_tickers[20:30]  # Replace with the desired tickers
final_df3 = scrape_multiple_stock_data(tickers)
final_df3

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$18.37,0.48\n(2.68%),$17.87\n-\n$18.88,"361,462","335,891","$6,634,489",$18.51,$18.10 - $18.51,$18.355,...,"645,984,181",Industrials,SVW / AIA / QAN,$0.60,$0.25,$2.484,0 Buy\n· 5 Hold\n· 12 Sell,Strong Sell,01/10/23,REH
1,Value,$29.25,0.61\n(2.13%),$28.75\n-\n$29.42,"814,627","800,978","$23,798,301",$29.15,$28.78 - $29.42,$29.215,...,"363,585,587",Industrials,AIA / REH / QAN,$1.677,$0.46,$4.643,10 Buy\n· 4 Hold\n· 0 Sell,Strong Buy,01/10/23,SVW
2,Value,$16.76,0.06\n(0.36%),$16.40\n-\n$16.98,"672,166","1,063,968","$11,276,621",$16.96,$16.71 - $16.96,$16.775,...,"526,751,357",Industrials,QAN / ALX / IFT,$0.07,$0.50,-$0.876,9 Buy\n· 3 Hold\n· 1 Sell,Buy,01/10/23,WOR
3,Value,$9.54,0.29\n(3.14%),$9.02\n-\n‐,"18,508","44,060","$174,664",$9.35,$9.32 - $9.54,$9.435,...,"831,889,987",Industrials,ALX / WOR / QAN,$0.432,$0.1782,$3.058,6 Buy\n· 5 Hold\n· 0 Sell,Buy,01/10/23,IFT
4,Value,$4.09,0.07\n(1.74%),$4.08\n-\n$4.12,"1,123,367","807,510","$4,591,552",$4.04,$4.03 - $4.115,$4.085,...,"783,043,596",Industrials,RWC / DOW / VNT,$0.284,$0.3162,$3.061,10 Buy\n· 3 Hold\n· 1 Sell,Buy,01/10/23,FBU
5,Value,$3.70,0.10\n(2.78%),$3.68\n-\n$3.72,"1,857,807","3,131,867","$6,864,044",$3.66,$3.66 - $3.71,$3.695,...,"790,094,765",Industrials,FBU / DOW / VNT,$0.177,$0.1424,$0.157,13 Buy\n· 7 Hold\n· 0 Sell,Buy,01/10/23,RWC
6,Value,$2.76,0.01\n(0.36%),$2.75\n-\n$2.77,"73,844,197","2,949,504","$200,600,620",$2.75,$2.72 - $2.78,$2.755,...,"855,484,445",Industrials,BIN / DOW / AIZ,$0.202,$0.1659,-$0.762,9 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,VNT
7,Value,$6.41,0.06\n(0.95%),$6.32\n-\n$6.46,"649,066","557,056","$4,153,817",$6.40,$6.34 - $6.46,$6.40,...,"276,315,537",Industrials,IPH / KLS / APM,$0.179,$0.09,$0.274,11 Buy\n· 3 Hold\n· 1 Sell,Buy,01/10/23,JLG
8,Value,$14.23,0.01\n(-0.07%),$13.80\n-\n$14.39,"163,650","202,748","$2,333,920",$14.30,$14.14 - $14.40,$14.26,...,"96,913,881",Industrials,MAD / DBI / RDX,$0.55,$0.49,$0.617,9 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,MND
9,Value,$2.34,0.00\n(0.00),$2.29\n-\n$2.34,"20,499","276,203","$47,788",$2.31,$2.29 - $2.34,$2.33,...,"525,081,428",Industrials,MMS / SIQ / NWH,$38.768,$0.00,$0.447,5 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,RDX


In [17]:
# Output 4th 10 tickers
tickers = all_tickers[30:40]  # Replace with the desired tickers
final_df4 = scrape_multiple_stock_data(tickers)
final_df4

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Ticker,Breakdown,Recommendation,Last Updated
0,Value,$0.285,0.025\n(9.62%),$0.27\n-\n$0.285,"110,064","191,029","$30,873",$0.265,$0.265 - $0.29,$0.2805,...,"51,907,925",Basic Materials,ADV / ASE / MRZ,-$0.044,$0.00,$0.123,SLB,NaN,NaN,NaN
1,Value,$0.115,0.005\n(-4.17%),$0.11\n-\n$0.12,"235,426","99,197","$27,978",$0.12,$0.115 - $0.12,$0.119,...,"133,865,666",Basic Materials,SGA / MMA / AII,-$0.067,$0.00,$0.034,TRE,NaN,NaN,NaN
2,Value,$0.043,0.00\n(0.00),$0.039\n-\n$0.043,"66,652","67,243","$2,866",$0.043,$0.043 - $0.043,$0.043,...,"195,415,646",Industrials,14D / TTI / 1CG,-$0.107,$0.00,$0.048,OXT,NaN,NaN,NaN
3,Value,$0.09,0.00\n(0.00),$0.093\n-\n$0.115,0,"16,786",$0,‐,‐ - ‐,‐,...,"36,393,239",Basic Materials,CRS / GNM / CBY,-$0.116,$0.00,$0.123,S3N,NaN,NaN,NaN
4,Value,$0.079,0.01\n(-11.24%),$0.079\n-\n$0.095,"35,572","129,659","$2,983",$0.086,$0.079 - $0.086,$0.084,...,"60,487,930",Basic Materials,THR / BOA / SNX,$0.00,$0.00,$0.00,HRE,NaN,NaN,NaN
5,Value,$0.063,0.001\n(-1.56%),$0.063\n-\n$0.066,"265,082","109,818","$16,917",$0.064,$0.063 - $0.064,$0.064,...,"59,575,058",Basic Materials,ACS / OM1 / KAL,-$0.081,$0.00,$0.077,KLI,NaN,NaN,NaN
6,Value,$0.10,0.011\n(12.36%),$0.096\n-\n$0.10,"193,111","56,492","$18,534",$0.094,$0.094 - $0.10,$0.096,...,"38,562,492",Basic Materials,ENV / NCR / ADG,$0.00,$0.00,$0.00,DY6,NaN,NaN,NaN
7,Value,$13.45,0.14\n(1.05%),$13.40\n-\n$13.47,"4,749,010","3,860,511","$63,655,591",$13.26,$13.11 - $13.50,$13.415,...,"1,391,408,373",Industrials,CPU / SYD / REH,$0.464,$0.395,$1.145,BXB,12 Buy\n· 5 Hold\n· 2 Sell,Buy,01/10/23
8,Value,$25.13,0.35\n(1.41%),$24.79\n-\n$25.30,"1,529,083","1,430,562","$38,417,867",$25.04,$24.98 - $25.20,$25.125,...,"603,729,336",Industrials,REH / BXB / SVW,$0.735,$0.70,-$1.908,CPU,11 Buy\n· 3 Hold\n· 0 Sell,Strong Buy,01/10/23
9,Value,$10.94,0.17\n(1.58%),$10.68\n-\n$11.00,"1,181,048","751,589","$12,910,551",$10.91,$10.87 - $10.99,$10.94,...,"484,167,684",Industrials,CWY / QUB / AZJ,$0.566,$0.397,-$0.097,ALQ,8 Buy\n· 5 Hold\n· 0 Sell,Buy,01/10/23


In [18]:
# Output 5th 10 tickers
tickers = all_tickers[40:50]  # Replace with the desired tickers
final_df5 = scrape_multiple_stock_data(tickers)
final_df5

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$2.39,0.09\n(3.91%),$2.35\n-\n$2.40,"5,042,306","4,298,610","$11,977,377",$2.32,$2.30 - $2.40,$2.375,...,"2,228,434,483",Industrials,ALQ / QUB / AZJ,$0.01,$0.049,-$0.059,8 Buy\n· 7 Hold\n· 2 Sell,Buy,01/10/23,CWY
1,Value,$3.78,0.01\n(0.27%),$3.76\n-\n$3.80,"2,061,252","1,366,844","$7,811,335",$3.82,$3.75 - $3.845,$3.79,...,"671,573,679",Industrials,VNT / BIN / RWC,-$0.59,$0.13,$0.163,5 Buy\n· 5 Hold\n· 0 Sell,Buy,01/10/23,DOW
2,Value,$2.09,0.01\n(-0.48%),$2.07\n-\n$2.11,"1,139,385","1,143,498","$2,390,561",$2.10,$2.08 - $2.13,$2.095,...,"917,181,946",Industrials,JLG / AIZ / IPH,$0.117,$0.10,-$0.773,7 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,APM
3,Value,$7.10,0.18\n(2.60%),$6.80\n-\n$7.15,"1,032,860","589,916","$7,327,336",$7.08,$6.96 - $7.13,$7.09,...,"240,021,838",Industrials,KLS / JLG / SLK,$0.284,$0.33,-$1.117,8 Buy\n· 5 Hold\n· 1 Sell,Buy,01/10/23,IPH
4,Value,$6.83,0.09\n(1.34%),$6.75\n-\n$6.90,"331,916","171,289","$2,264,894",$6.80,$6.77 - $7.13,$6.87,...,"200,000,000",Industrials,MND / DBI / RDX,$0.182,$0.058,$0.598,1 Buy\n· 5 Hold\n· 5 Sell,Sell,01/10/23,MAD
5,Value,$17.60,0.14\n(0.80%),$17.57\n-\n$18.06,"321,679","237,302","$5,710,750",$17.57,$17.57 - $18.03,$17.755,...,"69,643,024",Industrials,RDX / SIQ / NWH,$0.89,$1.98,$1.148,5 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,MMS
6,Value,$8.94,0.01\n(-0.11%),$8.80\n-\n$8.95,"447,619","455,704","$3,993,171",$8.97,$8.835 - $9.04,$8.92,...,"132,836,669",Industrials,NWH / MMS / RDX,$0.438,$0.445,-$0.366,7 Buy\n· 3 Hold\n· 1 Sell,Buy,01/10/23,SIQ
7,Value,$52.32,0.89\n(1.73%),$52.00\n-\n$52.35,"1,362,196","1,474,727","$71,096,006",$52.10,$51.86 - $52.355,$52.23,...,"1,133,992,429",Consumer Cyclical,ALL / TLC / CWN,$2.176,$1.91,$3.163,10 Buy\n· 6 Hold\n· 3 Sell,Buy,01/10/23,WES
8,Value,$46.58,0.71\n(1.55%),$46.20\n-\n$46.59,"454,810","382,298","$21,116,905",$46.40,$46.19 - $46.58,$46.43,...,"109,333,981",Consumer Cyclical,DMP / HVN / FLT,$4.779,$3.12,$3.551,3 Buy\n· 6 Hold\n· 10 Sell,Sell,01/10/23,JBH
9,Value,$3.69,0.02\n(0.55%),$3.68\n-\n$3.72,"1,673,253","2,803,172","$6,193,601",$3.70,$3.67 - $3.725,$3.70,...,"1,246,006,654",Consumer Cyclical,DMP / FLT / JBH,$0.432,$0.305,$3.51,2 Buy\n· 7 Hold\n· 9 Sell,Sell,01/10/23,HVN


In [20]:
# Output 6th 10 tickers
tickers = all_tickers[50:60]  # Replace with the desired tickers
final_df6 = scrape_multiple_stock_data(tickers)
final_df6

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$23.52,0.22\n(0.94%),$23.50\n-\n$23.58,"190,239","229,196","$4,482,301",$23.62,$23.46 - $23.72,$23.56,...,"159,458,770",Consumer Cyclical,APE / FLT / BRG,$1.686,$1.49,$5.763,9 Buy\n· 5 Hold\n· 3 Sell,Buy,01/10/23,PMV
1,Value,$13.21,0.02\n(-0.15%),$13.10\n-\n$13.54,"315,396","371,873","$4,182,371",$13.50,$13.14 - $13.51,$13.26,...,"256,900,410",Consumer Cyclical,BRG / PMV / SUL,$1.083,$0.73,$1.359,11 Buy\n· 6 Hold\n· 2 Sell,Buy,01/10/23,APE
2,Value,$13.31,0.01\n(0.08%),$13.20\n-\n$13.34,"774,318","868,298","$10,311,787",$13.40,$13.23 - $13.40,$13.32,...,"225,826,500",Consumer Cyclical,BRG / APE / ARB,$1.154,$1.03,$2.308,3 Buy\n· 9 Hold\n· 8 Sell,Sell,01/10/23,SUL
3,Value,$18.48,0.08\n(0.44%),$18.40\n-\n$18.80,"337,637","270,259","$6,256,288",$18.69,$18.305 - $18.85,$18.53,...,"109,692,639",Consumer Cyclical,BAP / TAH / IVC,$0.619,$0.69,$0.69,12 Buy\n· 3 Hold\n· 2 Sell,Buy,01/10/23,LOV
4,Value,$5.54,0.07\n(-1.25%),$5.53\n-\n$5.60,"759,049","1,176,056","$4,248,839",$5.72,$5.54 - $5.72,$5.60,...,"339,412,500",Consumer Cyclical,TAH / IVC / LOV,$0.362,$0.22,$0.959,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,BAP
5,Value,$1.955,0.02\n(1.03%),$1.945\n-\n$1.96,"1,673,098","825,084","$3,260,834",$1.935,$1.935 - $1.975,$1.953,...,"563,053,196",Consumer Cyclical,CKF / CTT / PWH,$0.156,$0.175,$0.105,7 Buy\n· 5 Hold\n· 2 Sell,Buy,01/10/23,AX1
6,Value,$4.42,0.00\n(0.00),$4.42\n-\n$4.67,0,58,$0,‐,‐ - ‐,‐,...,"222,765,778",Consumer Cyclical,PWH / NCK / CTT,$0.384,$0.2662,$1.376,0 Buy\n· 4 Hold\n· 4 Sell,Sell,01/10/23,BGP
7,Value,$22.45,0.70\n(3.22%),$21.90\n-\n$22.52,"277,724","231,182","$6,240,655",$22.48,$22.30 - $22.65,$22.47,...,"143,425,348",Consumer Cyclical,APE / SUL / PMV,$0.766,$0.305,$2.584,7 Buy\n· 8 Hold\n· 2 Sell,Hold,01/10/23,BRG
8,Value,$8.41,0.41\n(-4.65%),$8.41\n-\n$8.45,"9,958","2,086","$84,321",$8.50,$8.40 - $8.50,$8.47,...,"29,210,100",Consumer Cyclical,PBH / UNI / BBN,-$0.377,$0.00,-$0.081,NaN,NaN,NaN,AKP
9,Value,$1.65,0.01\n(-0.60%),$1.65\n-\n$1.71,"23,572","87,944","$39,066",$1.66,$1.65 - $1.66,$1.6575,...,"94,284,579",Consumer Cyclical,MTO / RBL / RCT,$0.022,$0.021,$1.254,4 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,FWD


In [21]:
# Output 7th 10 tickers
tickers = all_tickers[60:70]  # Replace with the desired tickers
final_df7 = scrape_multiple_stock_data(tickers)
final_df7

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Ticker,Breakdown,Recommendation,Last Updated
0,Value,$4.20,0.00\n(0.00),$4.00\n-\n$4.20,0,568,$0,‐,‐ - ‐,‐,...,"35,184,767",Consumer Cyclical,RBL / MTO / FWD,$0.307,$0.10,$3.402,WAT,NaN,NaN,NaN
1,Value,$0.26,0.005\n(1.96%),$0.26\n-\n$0.28,"144,845","351,328","$37,365",$0.26,$0.25 - $0.26,$0.258,...,"496,635,761",Consumer Cyclical,GLB / STP / VVA,$0.045,$0.011,$0.846,AVJ,NaN,NaN,NaN
2,Value,$3.10,0.05\n(1.64%),$2.85\n-\n$3.09,526,"1,341","$1,631",$3.10,$3.10 - $3.10,$3.10,...,"41,463,818",Consumer Cyclical,AVJ / STP / VVA,$0.038,$0.07,$1.694,GLB,NaN,NaN,NaN
3,Value,$2.63,0.09\n(3.54%),$2.63\n-\n$2.68,"19,621","6,969","$51,593",$2.57,$2.57 - $2.65,$2.63,...,"35,436,742",Consumer Cyclical,FUN / BET / ABY,$0.06,$0.13,$0.749,TWD,NaN,NaN,NaN
4,Value,$0.845,0.00\n(0.00),$0.84\n-\n$0.845,"41,138","15,859","$34,729",$0.845,$0.84 - $0.845,$0.844,...,"96,415,670",Consumer Cyclical,JYC / SGH / BET,$0.101,$0.135,$0.707,SHM,NaN,NaN,NaN
5,Value,$0.18,0.03\n(20.00%),$0.145\n-\n$0.18,"164,711","5,199","$27,891",$0.15,$0.15 - $0.18,$0.1695,...,"359,906,450",Consumer Cyclical,CCX / KED / DSK,-$0.137,$0.00,$0.025,SIO,NaN,NaN,NaN
6,Value,$0.1875,0.0025\n(1.35%),$0.185\n-\n$0.19,"10,000","44,243","$1,888",$0.19,$0.1875 - $0.19,$0.189,...,"284,014,642",Consumer Cyclical,EBG / RDH / DSK,$0.013,$0.01,$0.291,GAP,NaN,NaN,NaN
7,Value,$39.32,0.22\n(0.56%),$39.10\n-\n$39.63,"782,289","1,195,642","$30,772,436",$39.80,$39.15 - $39.81,$39.335,...,"648,560,092",Consumer Cyclical,TLC / CWN / IEL,$1.641,$0.56,$3.287,ALL,13 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23
8,Value,$117.93,4.14\n(3.64%),‐\n-\n‐,"127,849","91,994","$15,083,912",$117.78,$116.24 - $117.93,$117.56,...,"14,626,044",Consumer Cyclical,IVC / SGR / TAH,$0.46,$0.00,-$31.426,LNW,10 Buy\n· 4 Hold\n· 0 Sell,Strong Buy,01/10/23
9,Value,$4.60,0.02\n(-0.43%),$4.57\n-\n$4.61,"2,926,372","3,740,443","$13,495,257",$4.68,$4.57 - $4.68,$4.61,...,"2,225,771,703",Consumer Cyclical,CWN / IEL / JBH,$0.119,$0.15,-$1.216,TLC,10 Buy\n· 8 Hold\n· 0 Sell,Buy,01/10/23


In [22]:
# Output 8th 10 tickers
tickers = all_tickers[70:80]  # Replace with the desired tickers
final_df8 = scrape_multiple_stock_data(tickers)
final_df8

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$23.20,0.46\n(2.02%),$23.18\n-\n$23.50,"623,214","992,837","$14,414,485",$23.30,$22.80 - $23.39,$23.13,...,"278,336,211",Consumer Cyclical,JBH / DMP / HVN,$0.533,$0.41,-$0.075,12 Buy\n· 3 Hold\n· 2 Sell,Buy,01/10/23,IEL
1,Value,$51.95,0.63\n(1.23%),$51.81\n-\n$52.00,"358,193","321,427","$18,683,487",$52.28,$51.80 - $52.85,$52.13,...,"89,808,518",Consumer Cyclical,HVN / JBH / FLT,$0.761,$1.10,-$7.52,8 Buy\n· 7 Hold\n· 3 Sell,Hold,01/10/23,DMP
2,Value,$19.07,0.16\n(0.85%),$19.00\n-\n$19.08,"548,906","984,113","$10,463,238",$19.02,$18.93 - $19.175,$19.06,...,"219,427,708",Consumer Cyclical,HVN / PMV / DMP,$0.225,$0.18,$0.367,13 Buy\n· 6 Hold\n· 1 Sell,Buy,01/10/23,FLT
3,Value,$16.92,0.10\n(0.60%),$16.89\n-\n$16.95,"311,741","442,415","$5,289,803",$17.10,$16.87 - $17.18,$16.97,...,"146,325,746",Consumer Cyclical,WEB / ARB / LOV,$0.529,$0.28,$1.115,15 Buy\n· 4 Hold\n· 2 Sell,Buy,01/10/23,CTD
4,Value,$6.27,0.08\n(1.29%),$6.25\n-\n$6.29,"1,048,677","1,355,905","$6,597,155",$6.30,$6.26 - $6.34,$6.29,...,"385,982,154",Consumer Cyclical,CTD / ARB / LOV,$0.038,$0.00,$0.082,12 Buy\n· 6 Hold\n· 1 Sell,Buy,01/10/23,WEB
5,Value,$0.815,0.00\n(0.00),$0.81\n-\n$0.825,"3,166,111","6,794,018","$2,587,912",$0.825,$0.81 - $0.83,$0.8175,...,"2,282,410,989",Consumer Cyclical,BAP / IVC / LNW,$0.029,$0.023,-$0.213,9 Buy\n· 5 Hold\n· 1 Sell,Buy,01/10/23,TAH
6,Value,$12.67,0.01\n(0.08%),$12.66\n-\n$12.69,"2,312,768","498,571","$29,311,710",$12.68,$12.66 - $12.69,$12.675,...,"144,060,733",Consumer Cyclical,TAH / BAP / LNW,$0.135,$0.11,$2.157,0 Buy\n· 7 Hold\n· 4 Sell,Sell,01/10/23,IVC
7,Value,$35.50,0.40\n(1.14%),$35.50\n-\n$35.54,"1,874,760","1,876,943","$66,488,070",$35.38,$35.18 - $35.60,$35.435,...,"1,221,584,029",Consumer Defensive,COL / CCL / EDV,$1.323,$1.04,$0.601,10 Buy\n· 6 Hold\n· 3 Sell,Buy,01/10/23,WOW
8,Value,$15.45,0.12\n(0.78%),$15.39\n-\n$15.47,"2,191,862","2,769,498","$33,877,362",$15.57,$15.38 - $15.57,$15.455,...,"1,338,391,534",Consumer Defensive,CCL / EDV / TWE,$0.779,$0.66,$0.987,3 Buy\n· 10 Hold\n· 8 Sell,Sell,01/10/23,COL
9,Value,$5.01,0.08\n(1.62%),$5.00\n-\n$5.03,"4,089,519","4,594,666","$20,540,227",$4.97,$4.94 - $5.06,$5.015,...,"1,790,980,017",Consumer Defensive,CCL / TWE / MTS,$0.295,$0.293,-$0.308,7 Buy\n· 6 Hold\n· 3 Sell,Hold,01/10/23,EDV


In [23]:
# Output 9th 10 tickers
tickers = all_tickers[80:90]  # Replace with the desired tickers
final_df9 = scrape_multiple_stock_data(tickers)
final_df9

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$3.74,0.02\n(0.54%),$3.73\n-\n$3.76,"2,821,272","2,448,800","$10,560,793",$3.74,$3.72 - $3.775,$3.745,...,"977,060,067",Consumer Defensive,A2M / BKL / GNC,$0.267,$0.225,$0.183,8 Buy\n· 5 Hold\n· 2 Sell,Buy,01/10/23,MTS
1,Value,$7.14,0.10\n(1.42%),$7.06\n-\n$7.15,"403,841","484,478","$2,888,824",$7.15,$7.11 - $7.22,$7.155,...,"224,336,997",Consumer Defensive,UMG / CGC / ING,$1.487,$0.54,$6.034,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,GNC
2,Value,$0.066,0.003\n(4.76%),$0.066\n-\n$0.07,"454,964","161,094","$29,583",$0.065,$0.065 - $0.066,$0.065,...,"393,359,730",Consumer Defensive,WNX / BXN / SFG,-$0.004,$0.00,-$0.17,NaN,NaN,NaN,MMM
3,Value,$34.01,0.16\n(0.47%),$34.00\n-\n$34.05,"4,262,254","5,111,264","$145,184,300",$34.10,$33.81 - $34.28,$34.01,...,"1,898,749,771",Energy,WPL / STO / OSH,$3.734,$3.3974,$16.707,8 Buy\n· 7 Hold\n· 4 Sell,Hold,01/10/23,WDS
4,Value,$7.42,0.09\n(-1.20%),$7.39\n-\n$7.45,"10,850,167","8,129,617","$80,681,152",$7.59,$7.36 - $7.60,$7.435,...,"3,247,772,961",Energy,WPL / OSH / ALD,$0.523,$0.3576,$4.204,15 Buy\n· 4 Hold\n· 0 Sell,Strong Buy,01/10/23,STO
5,Value,$33.08,0.33\n(1.01%),$32.90\n-\n$33.20,"664,103","750,220","$21,967,524",$33.19,$32.96 - $33.25,$33.08,...,"238,302,099",Energy,OSH / YAL / WHC,$0.535,$2.50,$8.53,11 Buy\n· 3 Hold\n· 0 Sell,Strong Buy,01/10/23,ALD
6,Value,$4.71,0.07\n(-1.46%),$4.70\n-\n$4.72,"1,437,445","1,718,133","$6,783,177",$4.80,$4.68 - $4.84,$4.72,...,"1,320,439,437",Energy,WHC / NHC / ALD,$2.124,$1.07,$5.979,4 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,YAL
7,Value,$7.15,0.14\n(-1.92%),$7.15\n-\n$7.16,"4,881,885","8,099,005","$35,087,164",$7.33,$7.135 - $7.38,$7.185,...,"836,600,784",Energy,YAL / NHC / VEA,$3.028,$0.74,$6.273,6 Buy\n· 8 Hold\n· 2 Sell,Hold,01/10/23,WHC
8,Value,$9.69,0.16\n(1.68%),$9.64\n-\n$9.85,"1,887","34,016","$18,508",$9.60,$9.60 - $9.85,$9.81,...,"26,447,035",Energy,HZN / CVN / COE,-$0.14,$0.00,$0.849,15 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,NXG
9,Value,$5.43,0.04\n(-0.73%),$5.43\n-\n$5.52,"3,631,956","3,651,414","$19,665,217",$5.53,$5.38 - $5.57,$5.415,...,"845,335,464",Energy,VEA / BPT / WHC,$1.186,$0.96,$2.902,0 Buy\n· 4 Hold\n· 7 Sell,Strong Sell,01/10/23,NHC


In [24]:
# Output 10th 10 tickers
tickers = all_tickers[90:100]  # Replace with the desired tickers
final_df10 = scrape_multiple_stock_data(tickers)
final_df10

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$2.88,0.03\n(1.05%),$2.85\n-\n$2.89,"10,554,541","4,478,451","$30,358,913",$2.89,$2.84 - $2.89,$2.87,...,"1,544,166,247",Energy,NHC / BPT / PDN,-$0.056,$0.218,$1.141,7 Buy\n· 7 Hold\n· 0 Sell,Buy,01/10/23,VEA
1,Value,$1.525,0.02\n(-1.29%),$1.525\n-\n$1.53,"5,569,880","6,835,981","$8,542,828",$1.575,$1.525 - $1.575,$1.534,...,"2,281,333,656",Energy,PDN / VEA / NHC,$0.176,$0.04,$1.666,12 Buy\n· 5 Hold\n· 3 Sell,Buy,01/10/23,BPT
2,Value,$1.005,0.015\n(-1.47%),$1.00\n-\n$1.01,"21,560,872","17,898,358","$21,900,947",$1.04,$1.00 - $1.05,$1.016,...,"2,983,375,533",Energy,BPT / ZEL / BOE,-$0.004,$0.00,$0.031,8 Buy\n· 2 Hold\n· 1 Sell,Buy,01/10/23,PDN
3,Value,$22.04,0.37\n(1.71%),$22.01\n-\n$22.15,"4,016,206","3,131,924","$88,933,123",$22.49,$21.95 - $22.58,$22.145,...,"1,899,182,071",Real Estate,SCG / SGP / VCX,$0.813,$0.30,$9.043,14 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,GMG
4,Value,$2.58,0.06\n(2.38%),$2.56\n-\n$2.59,"11,435,568","10,571,001","$29,482,262",$2.57,$2.56 - $2.60,$2.58,...,"5,190,378,339",Real Estate,SGP / VCX / MGR,-$0.006,$0.165,$3.57,10 Buy\n· 3 Hold\n· 2 Sell,Buy,01/10/23,SCG
5,Value,$4.18,0.27\n(6.91%),$4.06\n-\n$4.20,"555,107","234,822","$2,323,896",$4.16,$4.16 - $4.21,$4.185,...,"152,090,780",Real Estate,PWG / QAL / RFF,-$0.346,$0.00,$6.213,9 Buy\n· 8 Hold\n· 5 Sell,Hold,01/10/23,URW
6,Value,$3.73,0.07\n(1.91%),$3.72\n-\n$3.75,"10,141,246","6,461,825","$37,790,904",$3.70,$3.69 - $3.75,$3.73,...,"2,387,171,662",Real Estate,VCX / MGR / GPT,$0.182,$0.262,$2.243,7 Buy\n· 4 Hold\n· 3 Sell,Hold,01/10/23,SGP
7,Value,$1.795,0.025\n(1.41%),$1.79\n-\n$1.805,"18,463,593","12,035,124","$33,267,610",$1.81,$1.795 - $1.825,$1.801,...,"4,552,275,358",Real Estate,MGR / GPT / SGP,$0.06,$0.12,$2.301,1 Buy\n· 11 Hold\n· 6 Sell,Sell,01/10/23,VCX
8,Value,$1.945,0.04\n(2.10%),$1.93\n-\n$1.95,"14,662,763","11,920,309","$28,638,387",$1.95,$1.92 - $1.97,$1.9475,...,"3,945,860,217",Real Estate,GPT / DXS / VCX,-$0.042,$0.105,$2.656,3 Buy\n· 5 Hold\n· 8 Sell,Sell,01/10/23,MGR
9,Value,$3.89,0.08\n(2.10%),$3.86\n-\n$3.90,"7,198,814","4,961,637","$27,992,625",$3.92,$3.85 - $3.93,$3.89,...,"1,915,577,430",Real Estate,DXS / MGR / VCX,-$0.033,$0.248,$5.978,8 Buy\n· 3 Hold\n· 4 Sell,Hold,01/10/23,GPT


In [25]:
# Output 11th 10 tickers
tickers = all_tickers[100:110]  # Replace with the desired tickers
final_df11 = scrape_multiple_stock_data(tickers)
final_df11

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$6.89,0.17\n(2.53%),$6.84\n-\n$6.94,"2,937,146","3,542,683","$20,248,239",$6.89,$6.85 - $6.94,$6.895,...,"1,075,565,246",Real Estate,GPT / MGR / VCX,-$0.637,$0.516,$10.771,9 Buy\n· 4 Hold\n· 2 Sell,Buy,01/10/23,DXS
1,Value,$9.77,0.35\n(3.72%),$9.65\n-\n$9.86,"1,851,170","1,554,849","$18,025,975",$9.69,$9.54 - $9.81,$9.74,...,"472,997,199",Real Estate,LLC / SCP / NSR,$0.407,$0.4254,$6.642,9 Buy\n· 5 Hold\n· 1 Sell,Buy,01/10/23,CHC
2,Value,$2.06,0.03\n(1.48%),$2.04\n-\n$2.08,"4,216,672","2,726,955","$8,689,761",$2.07,$2.04 - $2.08,$2.06,...,"1,361,129,474",Real Estate,SCP / RGN / CLW,$0.258,$0.11,$0.166,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,NSR
3,Value,$163.24,2.85\n(1.78%),$163.00\n-\n$163.30,"1,915,313","803,831","$308,904,600",$157.20,$155.30 - $163.46,$160.97,...,"386,476,754",Financial Services,WBC / ANZ / NAB,$13.163,$7.50,$75.886,11 Buy\n· 6 Hold\n· 0 Sell,Buy,01/10/23,MQG
4,Value,$81.18,16.33\n(25.18%),$80.50\n-\n$81.40,"931,761","129,214","$70,065,024",$78.32,$78.32 - $81.50,$80.22,...,"33,781,917",Financial Services,HUB / VEU / ETHI,-$0.43,$0.00,$5.314,36 Buy\n· 13 Hold\n· 2 Sell,Buy,01/10/23,SQ2
5,Value,$34.35,0.46\n(1.36%),$34.00\n-\n$34.45,"390,045","379,902","$13,412,666",$34.28,$34.08 - $34.59,$34.385,...,"360,967,863",Financial Services,VAS / ASX / IAG,$2.186,$0.94,$23.443,4 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,SOL
6,Value,$57.04,0.11\n(0.19%),$56.80\n-\n$57.40,"526,613","399,858","$30,056,365",$57.49,$56.59 - $57.49,$57.075,...,"193,595,162",Financial Services,SOL / VAS / MPL,$1.639,$2.283,$1.782,2 Buy\n· 7 Hold\n· 8 Sell,Sell,01/10/23,ASX
7,Value,$38.40,1.15\n(3.09%),$37.60\n-\n$38.42,"301,074","44,414","$11,449,370",$38.01,$37.60 - $38.40,$38.03,...,"8,583,904",Financial Services,MYS / WXOZ / VAE,$2.03,$2.3479,$4.177,0 Buy\n· 7 Hold\n· 9 Sell,Strong Sell,01/10/23,JHG
8,Value,$6.00,0.02\n(0.33%),$5.97\n-\n$6.11,"1,118,381","1,431,728","$6,705,903",$6.00,$5.97 - $6.09,$5.995,...,"690,869,975",Financial Services,GQG / QUAL / IOZ,$0.39,$0.24,$5.172,9 Buy\n· 6 Hold\n· 1 Sell,Buy,01/10/23,CGF
9,Value,$1.37,0.03\n(2.24%),$1.35\n-\n$1.38,"1,982,208","3,047,845","$2,704,071",$1.36,$1.345 - $1.375,$1.3645,...,"2,952,805,434",Financial Services,QUAL / CGF / IOZ,$0.081,$0.0831,$0.105,8 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,GQG


In [26]:
# Output 12th 10 tickers
tickers = all_tickers[110:120]  # Replace with the desired tickers
final_df12 = scrape_multiple_stock_data(tickers)
final_df12

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$13.45,0.15\n(1.13%),$13.30\n-\n$13.92,"362,007","452,800","$4,875,834",$13.50,$13.36 - $13.70,$13.47,...,"244,026,358",Financial Services,AAA / A200 / NDQ,$0.243,$0.24,$0.478,8 Buy\n· 6 Hold\n· 3 Sell,Hold,01/10/23,NWL
1,Value,$1.045,0.005\n(0.48%),$1.04\n-\n$1.05,"6,486,602","10,543,204","$6,798,941",$1.055,$1.04 - $1.06,$1.048,...,"2,750,784,831",Financial Services,IOO / AUB / VHY,-$0.014,$0.05,$1.329,1 Buy\n· 7 Hold\n· 6 Sell,Sell,01/10/23,AMP
2,Value,$32.25,0.47\n(1.48%),$31.85\n-\n$32.50,"162,990","259,280","$5,252,411",$31.80,$31.53 - $32.39,$32.225,...,"81,349,338",Financial Services,VEU / SQ2 / ETHI,$0.461,$0.325,$0.629,7 Buy\n· 8 Hold\n· 1 Sell,Buy,01/10/23,HUB
3,Value,$11.04,1.06\n(-8.76%),$11.02\n-\n$11.19,"10,019,715","2,147,718","$110,902,455",$11.35,$10.75 - $11.44,$11.045,...,"721,848,176",Consumer Defensive,EDV / CCL / MTS,$0.473,$0.35,$3.339,14 Buy\n· 3 Hold\n· 3 Sell,Buy,01/10/23,TWE
4,Value,$4.00,0.08\n(2.04%),$4.00\n-\n$4.03,"2,613,072","2,330,715","$10,417,738",$3.94,$3.94 - $4.02,$3.985,...,"722,934,808",Consumer Defensive,MTS / BKL / GNC,$0.211,$0.00,$1.437,5 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,A2M
5,Value,$3.14,0.04\n(1.29%),$3.11\n-\n$3.14,"2,242,742","3,106,760","$7,016,753",$3.11,$3.10 - $3.14,$3.13,...,"464,709,793",Consumer Defensive,ING / UMG / GNC,$0.046,$0.05,$1.045,7 Buy\n· 7 Hold\n· 0 Sell,Buy,01/10/23,CGC
6,Value,$3.92,0.01\n(0.26%),$3.91\n-\n$3.94,"3,193,935","1,778,755","$12,513,237",$3.94,$3.88 - $3.95,$3.92,...,"371,679,601",Consumer Defensive,CGC / UMG / GNC,$0.037,$0.145,$0.543,10 Buy\n· 3 Hold\n· 1 Sell,Buy,01/10/23,ING
7,Value,$6.20,0.20\n(3.33%),$6.17\n-\n$6.24,"946,430","643,671","$5,804,272",$6.08,$6.01 - $6.24,$6.135,...,"156,476,574",Consumer Defensive,BGA / TGR / AHY,$0.77,$0.51,$3.135,7 Buy\n· 7 Hold\n· 1 Sell,Buy,01/10/23,ELD
8,Value,$2.80,0.00\n(0.00),$2.80\n-\n$2.81,"1,367,843","827,132","$3,830,357",$2.83,$2.78 - $2.83,$2.80,...,"304,517,775",Consumer Defensive,AHY / AAC / ELD,-$0.756,$0.075,$1.771,0 Buy\n· 8 Hold\n· 7 Sell,Sell,01/10/23,BGA
9,Value,$1.275,0.005\n(0.39%),$1.265\n-\n$1.31,"475,822","215,816","$605,844",$1.27,$1.245 - $1.30,$1.2735,...,"602,766,747",Consumer Defensive,AHY / RIC / BGA,$0.008,$0.00,$2.57,4 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,AAC


In [27]:
# Output 13th 10 tickers
tickers = all_tickers[120:130]  # Replace with the desired tickers
final_df13 = scrape_multiple_stock_data(tickers)
final_df13

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$2.22,0.01\n(-0.45%),$2.19\n-\n$2.26,"398,801","512,985","$890,739",$2.23,$2.21 - $2.26,$2.235,...,"315,832,713",Consumer Defensive,AAC / AHY / BGA,$0.131,$0.0825,$0.854,7 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,RIC
1,Value,$1.29,0.01\n(0.78%),$1.29\n-\n$1.32,"317,234","236,616","$407,065",$1.285,$1.23 - $1.32,$1.283,...,"415,773,189",Consumer Defensive,SHV / HUO / SGLLV,$0.018,$0.033,$0.676,6 Buy\n· 1 Hold\n· 1 Sell,Buy,01/10/23,CBO
2,Value,$4.07,0.16\n(4.09%),$4.00\n-\n$4.20,"211,415","180,717","$855,441",$3.90,$3.90 - $4.10,$4.045,...,"121,058,664",Consumer Defensive,CBO / HUO / SGLLV,-$0.775,$0.02,$3.579,6 Buy\n· 2 Hold\n· 1 Sell,Buy,01/10/23,SHV
3,Value,$23.15,0.29\n(-1.24%),$23.12\n-\n$23.44,"3,827,691","3,549,122","$89,319,231",$23.42,$23.13 - $23.60,$23.345,...,"566,292,300",Healthcare,FPH / RHC / SHL,$0.615,$0.1894,$0.549,10 Buy\n· 7 Hold\n· 0 Sell,Buy,01/10/23,RMD
4,Value,$247.58,1.65\n(0.67%),$245.70\n-\n$248.52,"104,162","140,143","$25,796,002",$247.27,$245.68 - $249.17,$247.655,...,"65,533,900",Healthcare,SHL / RMD / FPH,$4.561,$3.30,$0.097,3 Buy\n· 12 Hold\n· 9 Sell,Sell,01/10/23,COH
5,Value,$29.52,0.32\n(1.10%),$29.41\n-\n$29.59,"895,079","979,937","$26,407,444",$29.43,$29.30 - $29.63,$29.505,...,"474,429,611",Healthcare,RMD / COH / FPH,$1.45,$1.04,-$0.112,10 Buy\n· 6 Hold\n· 4 Sell,Hold,01/10/23,SHL
6,Value,$20.14,0.41\n(2.08%),$19.90\n-\n$20.17,"271,490","326,399","$5,464,069",$19.90,$19.79 - $20.27,$20.125,...,"582,170,347",Healthcare,RHC / RMD / SHL,$0.43,$0.3762,$2.878,1 Buy\n· 9 Hold\n· 7 Sell,Sell,01/10/23,FPH
7,Value,$50.70,0.41\n(0.82%),$50.04\n-\n$50.77,"407,709","404,744","$20,651,647",$50.59,$50.31 - $51.17,$50.66,...,"229,434,689",Healthcare,FPH / RMD / SHL,$1.164,$0.75,-$8.756,6 Buy\n· 13 Hold\n· 1 Sell,Hold,01/10/23,RHC
8,Value,$81.67,1.78\n(2.23%),$81.60\n-\n$82.48,"118,951","164,521","$9,725,630",$81.09,$80.60 - $82.38,$81.76,...,"104,455,279",Healthcare,EBO / RHC / FPH,$0.58,$0.30,$1.124,5 Buy\n· 10 Hold\n· 1 Sell,Hold,01/10/23,PME
9,Value,$33.96,0.16\n(0.47%),$33.80\n-\n$34.00,"10,469","18,946","$356,307",$34.28,$33.80 - $34.28,$34.035,...,"191,807,695",Healthcare,PME / TLX / ANN,$1.446,$0.9093,-$0.325,6 Buy\n· 5 Hold\n· 1 Sell,Buy,01/10/23,EBO


In [28]:
# Output 14th 10 tickers
tickers = all_tickers[130:140]  # Replace with the desired tickers
final_df14 = scrape_multiple_stock_data(tickers)
final_df14

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$22.15,0.40\n(1.84%),$22.00\n-\n$22.16,"259,172","234,605","$5,721,887",$21.82,$21.82 - $22.18,$22.08,...,"125,771,856",Healthcare,TLX / SNZ / NEU,$1.167,$0.6946,$2.107,0 Buy\n· 10 Hold\n· 6 Sell,Sell,01/10/23,ANN
1,Value,$8.90,0.02\n(0.23%),$8.90\n-\n$9.20,2,466,$18,$8.95,$8.90 - $8.95,$8.925,...,"234,281,382",Healthcare,NEU / ANN / NAN,$1.153,$0.2685,$9.375,6 Buy\n· 3 Hold\n· 0 Sell,Buy,01/10/23,SNZ
2,Value,$3.91,0.04\n(1.03%),$3.89\n-\n$3.93,"752,709","581,378","$2,945,450",$3.95,$3.83 - $4.03,$3.915,...,"302,607,616",Healthcare,HLS / PNV / NEU,$0.065,$0.00,$0.541,3 Buy\n· 7 Hold\n· 7 Sell,Sell,01/10/23,NAN
3,Value,$0.40,0.00\n(0.00),$0.40\n-\n$0.405,"17,501","185,870","$7,000",$0.40,$0.40 - $0.40,$0.40,...,"170,459,499",Consumer Defensive,CSS / DBF / GDA,$0.031,$0.023,$0.179,NaN,NaN,NaN,PTL
4,Value,$0.395,0.00\n(0.00),$0.39\n-\n$0.40,"92,577","45,439","$36,691",$0.40,$0.39 - $0.405,$0.3965,...,"143,949,141",Consumer Defensive,GDA / DBF / EHH,$0.02,$0.03,$0.211,NaN,NaN,NaN,MCP
5,Value,$0.12,0.00\n(0.00),$0.115\n-\n$0.125,"160,641","190,675","$19,277",$0.12,$0.12 - $0.12,$0.12,...,"163,013,394",Consumer Defensive,OJC / FRM / WLD,-$0.022,$0.00,$0.032,NaN,NaN,NaN,BIO
6,Value,$0.013,0.00\n(0.00),$0.011\n-\n$0.013,"300,000","453,684","$3,900",$0.013,$0.013 - $0.013,$0.013,...,"1,901,645,398",Consumer Defensive,WNX / SFG / FOD,-$0.001,$0.00,$0.001,NaN,NaN,NaN,BXN
7,Value,$0.11,0.00\n(0.00),$0.12\n-\n$0.20,0,"1,029",$0,‐,‐ - ‐,‐,...,"135,082,981",Consumer Defensive,NUC / BFC / E33,-$0.037,$0.00,$0.257,NaN,NaN,NaN,S66
8,Value,$0.027,0.002\n(8.00%),$0.026\n-\n$0.03,"48,933","41,330","$1,407",$0.025,$0.025 - $0.03,$0.029,...,"247,640,513",Consumer Defensive,YOW / AAP / EXL,-$0.088,$0.00,$0.022,NaN,NaN,NaN,HPC
9,Value,$0.009,0.00\n(0.00),$0.009\n-\n$0.01,"1,958,812","592,619","$16,646",$0.009,$0.008 - $0.009,$0.0085,...,"631,343,835",Consumer Defensive,1AG / AAP / EVE,-$0.02,$0.00,$0.014,NaN,NaN,NaN,EXL


In [29]:
# Output 15th 10 tickers
tickers = all_tickers[140:150]  # Replace with the desired tickers
final_df15 = scrape_multiple_stock_data(tickers)
final_df15

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Ticker,Breakdown,Recommendation,Last Updated
0,Value,$0.006,0.00\n(0.00),$0.006\n-\n$0.007,0,"531,319",$0,‐,‐ - ‐,‐,...,"589,486,095",Consumer Defensive,JJF / NZS / CCO,-$0.007,$0.00,$0.004,SKN,NaN,NaN,NaN
1,Value,$0.004,0.00\n(0.00),$0.003\n-\n$0.004,"125,250","1,811,945",$501,$0.004,$0.004 - $0.004,$0.004,...,"817,119,281",Consumer Defensive,NZS / JJF / SKN,-$0.016,$0.00,-$0.001,CCO,NaN,NaN,NaN
2,Value,$0.01,0.00\n(0.00),$0.009\n-\n$0.012,0,"138,590",$0,‐,‐ - ‐,‐,...,"278,800,067",Consumer Defensive,FFF / HLF / HVM,-$0.007,$0.00,$0.01,HCT,NaN,NaN,NaN
3,Value,$15.68,0.13\n(-0.82%),$15.50\n-\n$15.84,"3,318,064","3,078,372","$52,064,978",$15.94,$15.57 - $15.94,$15.69,...,"1,493,915,310",Financial Services,SUN / IAG / VAS,$0.668,$0.44,$1.819,QBE,16 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23
4,Value,$13.52,0.04\n(-0.30%),$13.40\n-\n$13.54,"2,425,976","2,415,184","$32,811,388",$13.74,$13.47 - $13.74,$13.525,...,"1,270,262,714",Financial Services,IAG / VAS / SOL,$0.885,$0.60,$6.304,SUN,15 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23
5,Value,$5.77,0.04\n(-0.69%),$5.72\n-\n$5.80,"3,828,550","5,630,267","$22,150,424",$5.83,$5.765 - $5.83,$5.785,...,"2,427,998,043",Financial Services,VAS / SOL / ASX,$0.322,$0.15,$1.243,IAG,7 Buy\n· 8 Hold\n· 2 Sell,Hold,01/10/23
6,Value,$3.46,0.02\n(0.58%),$3.45\n-\n$3.48,"2,741,059","5,847,753","$9,480,094",$3.45,$3.435 - $3.47,$3.46,...,"2,754,003,240",Financial Services,AFI / ASX / SOL,$0.186,$0.146,$0.638,MPL,8 Buy\n· 7 Hold\n· 0 Sell,Buy,01/10/23
7,Value,$5.52,0.05\n(0.91%),$5.40\n-\n$5.59,"2,544,358","1,641,391","$14,028,278",$5.49,$5.42 - $5.54,$5.515,...,"1,038,561,895",Financial Services,IVV / VGS / BEN,$0.184,$0.15,-$0.28,SDF,8 Buy\n· 7 Hold\n· 0 Sell,Buy,01/10/23
8,Value,$7.35,0.07\n(0.96%),$7.35\n-\n$7.50,"691,828","816,518","$5,087,973",$7.37,$7.30 - $7.40,$7.355,...,"484,276,755",Financial Services,BOQ / VTS / NDQ,$0.403,$0.28,$1.027,NHF,6 Buy\n· 9 Hold\n· 1 Sell,Buy,01/10/23
9,Value,$27.68,0.48\n(1.77%),$27.50\n-\n$27.84,"350,894","209,517","$9,731,976",$27.44,$27.23 - $27.92,$27.72,...,"108,405,620",Financial Services,VHY / GOLD / AMP,$0.651,$0.64,-$6.245,AUB,13 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23


In [30]:
# Output 16th 10 tickers
tickers = all_tickers[150:160]  # Replace with the desired tickers
final_df16 = scrape_multiple_stock_data(tickers)
final_df16

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$4.36,0.16\n(-3.54%),$4.25\n-\n$4.45,"707,680","63,207","$3,121,579",$4.43,$4.20 - $4.45,$4.345,...,"359,479,974",Financial Services,MFF / PPH / WAM,$0.157,$0.135,-$0.184,8 Buy\n· 3 Hold\n· 0 Sell,Strong Buy,01/10/23,PSI
1,Value,$0.53,0.01\n(1.92%),$0.52\n-\n$0.535,"116,413","106,991","$61,305",$0.535,$0.525 - $0.535,$0.5265,...,"661,038,648",Financial Services,F100 / ALI / WDIV,$0.033,$0.03,$0.692,NaN,NaN,NaN,CVW
2,Value,$1.505,0.00\n(0.00),$1.50\n-\n$1.55,0,"60,048",$0,‐,‐ - ‐,‐,...,"190,610,251",Financial Services,IHD / PLUS / GBND,$0.024,$0.02,$0.295,7 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,GDG
3,Value,$45.50,0.12\n(0.26%),$45.50\n-\n$45.52,"6,766,318","7,345,298","$308,576,546",$45.80,$45.43 - $45.88,$45.61,...,"5,068,739,787",Basic Materials,FMG / RIO / NCM,$2.547,$2.6143,$8.184,11 Buy\n· 9 Hold\n· 3 Sell,Buy,01/10/23,BHP
4,Value,$23.23,0.02\n(-0.09%),$23.15\n-\n$23.24,"4,598,296","4,429,937","$106,123,851",$23.39,$23.06 - $23.47,$23.26,...,"3,078,964,918",Basic Materials,RIO / NCM / NEM,$1.557,$1.75,$5.745,2 Buy\n· 3 Hold\n· 15 Sell,Strong Sell,01/10/23,FMG
5,Value,$58.94,0.69\n(1.19%),$58.50\n-\n$60.00,"324,292","851,714","$19,134,266",$59.26,$58.50 - $59.45,$59.005,...,"352,663,842",Basic Materials,NCM / JHX / S32,-$1.06,$0.00,$21.871,NaN,NaN,NaN,NEM
6,Value,$122.60,0.84\n(0.69%),$122.00\n-\n$122.68,"1,560,659","1,213,521","$190,221,819",$121.54,$121.52 - $123.15,$122.52,...,"371,216,214",Basic Materials,NCM / NEM / FMG,$5.27,$5.8738,$29.015,12 Buy\n· 5 Hold\n· 1 Sell,Buy,01/10/23,RIO
7,Value,$13.79,0.13\n(0.95%),$13.74\n-\n$13.79,"4,111,185","2,242,606","$56,310,980",$13.67,$13.63 - $13.79,$13.715,...,"641,567,279",Basic Materials,BSL / OZL / PMGOLD,$0.655,$0.7394,-$1.982,1 Buy\n· 6 Hold\n· 6 Sell,Sell,01/10/23,AMC
8,Value,$41.10,0.23\n(0.56%),$40.55\n-\n$41.80,"984,502","1,015,583","$40,348,177",$41.56,$40.55 - $41.61,$41.00,...,"437,772,838",Basic Materials,NEM / NCM / S32,$1.14,$0.00,$2.866,14 Buy\n· 3 Hold\n· 1 Sell,Strong Buy,01/10/23,JHX
9,Value,$3.32,0.00\n(0.00),$3.31\n-\n$3.33,"9,145,589","16,906,395","$30,433,783",$3.35,$3.31 - $3.36,$3.33,...,"4,529,258,568",Basic Materials,NST / JHX / MIN,-$0.038,$0.1232,$2.017,14 Buy\n· 5 Hold\n· 0 Sell,Strong Buy,01/10/23,S32


In [31]:
# Output 17th 10 tickers
tickers = all_tickers[160:170]  # Replace with the desired tickers
final_df17 = scrape_multiple_stock_data(tickers)
final_df17

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$11.62,0.04\n(0.35%),$11.60\n-\n$11.62,"4,386,552","5,248,814","$50,875,209",$11.58,$11.47 - $11.67,$11.585,...,"1,149,221,116",Basic Materials,MIN / S32 / PLS,$0.503,$0.265,$7.315,10 Buy\n· 8 Hold\n· 0 Sell,Buy,01/10/23,NST
1,Value,$61.07,1.42\n(2.38%),$61.00\n-\n$61.60,"780,159","961,879","$47,903,241",$61.35,$61.07 - $61.94,$61.40,...,"195,186,428",Basic Materials,PLS / NST / PMGOLD,$3.805,$1.90,$17.689,13 Buy\n· 4 Hold\n· 3 Sell,Buy,01/10/23,MIN
2,Value,$3.75,0.11\n(3.02%),$3.74\n-\n$3.75,"32,842,788","31,521,298","$124,051,190",$3.77,$3.71 - $3.85,$3.775,...,"3,009,462,660",Basic Materials,MIN / PMGOLD / OZL,$0.789,$0.25,$1.126,11 Buy\n· 8 Hold\n· 2 Sell,Buy,01/10/23,PLS
3,Value,$34.36,0.41\n(1.21%),$33.00\n-\n$36.08,"69,788","224,082","$2,403,658",$34.47,$34.36 - $34.75,$34.44,...,"28,450,066",Communication Services,TUA / ABB / OML,$0.26,$0.2133,$0.763,11 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,NWS
4,Value,$32.00,0.26\n(-0.81%),‐\n-\n‐,175,"13,181","$5,628",$32.25,$32.00 - $32.25,$32.16,...,"3,864,003",N/A,-,$0.26,$0.2133,$0.763,NaN,NaN,NaN,NWSLV
5,Value,$155.40,3.31\n(2.18%),$154.00\n-\n$156.65,"189,854","181,347","$29,490,876",$155.00,$153.48 - $156.48,$155.335,...,"132,117,217",Communication Services,CAR / TPG / SPK,$2.695,$1.58,$4.479,3 Buy\n· 9 Hold\n· 7 Sell,Sell,01/10/23,REA
6,Value,$28.65,0.09\n(0.32%),$28.15\n-\n$28.97,"960,407","1,226,651","$27,602,775",$29.02,$28.57 - $29.13,$28.74,...,"377,017,360",Communication Services,TPG / SPK / SEK,$1.811,$0.61,-$2.949,13 Buy\n· 7 Hold\n· 0 Sell,Buy,01/10/23,CAR
7,Value,$21.80,0.16\n(0.74%),$21.60\n-\n$22.18,"986,774","840,007","$21,538,807",$21.89,$21.65 - $22.04,$21.83,...,"356,220,190",Communication Services,SPK / TPG / CAR,$0.564,$0.47,$0.19,10 Buy\n· 5 Hold\n· 2 Sell,Buy,01/10/23,SEK
8,Value,$1.915,0.035\n(1.86%),$1.905\n-\n$1.925,"1,942,500","2,908,145","$3,722,481",$1.91,$1.9025 - $1.93,$1.9165,...,"1,626,270,594",Communication Services,CNU / VOC / UWL,$0.11,$0.11,-$0.471,9 Buy\n· 5 Hold\n· 0 Sell,Buy,01/10/23,NEC
9,Value,$3.60,0.00\n(0.00),$3.58\n-\n$3.62,"1,181,415","723,777","$4,299,739",$3.63,$3.60 - $3.71,$3.64,...,"631,657,153",Communication Services,EVT / CNU / NEC,$0.058,$0.06,-$0.449,1 Buy\n· 10 Hold\n· 6 Sell,Sell,01/10/23,DHG


In [32]:
# Output 18th 10 tickers
tickers = all_tickers[170:180]  # Replace with the desired tickers
final_df18 = scrape_multiple_stock_data(tickers)
final_df18

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$10.85,0.13\n(1.21%),$10.75\n-\n$11.00,"142,629","118,664","$1,552,445",$10.93,$10.77 - $11.04,$10.885,...,"162,275,357",Communication Services,DHG / NWS / TUA,$0.655,$0.46,$2.347,6 Buy\n· 4 Hold\n· 0 Sell,Buy,01/10/23,EVT
1,Value,$1.345,0.03\n(2.28%),$1.31\n-\n$1.39,"265,533","602,027","$355,796",$1.33,$1.325 - $1.355,$1.34,...,"538,781,286",Communication Services,OPC / MNF / ABB,$0.054,$0.0475,$0.112,6 Buy\n· 4 Hold\n· 0 Sell,Buy,01/10/23,OML
2,Value,$0.27,0.005\n(1.89%),$0.265\n-\n$0.27,"1,044,006","1,128,745","$281,155",$0.27,$0.265 - $0.275,$0.2695,...,"1,553,571,241",Communication Services,HTA / SKT / HT1,$0.092,$0.00,-$0.216,1 Buy\n· 9 Hold\n· 6 Sell,Sell,01/10/23,SWM
3,Value,$6.88,0.10\n(1.48%),$6.87\n-\n$6.93,"445,370","389,981","$3,052,915",$6.79,$6.79 - $6.93,$6.855,...,"1,246,549,445",Financial Services,MPL / ARG / MGOC,$0.00,$0.25,$0.00,NaN,NaN,NaN,AFI
4,Value,$8.47,0.07\n(0.83%),$8.42\n-\n$8.50,"272,957","236,408","$2,312,000",$8.45,$8.45 - $8.51,$8.47,...,"758,559,349",Financial Services,MGOC / VGS / IVV,$0.00,$0.345,$0.00,NaN,NaN,NaN,ARG
5,Value,$1.78,0.01\n(0.57%),$1.78\n-\n$1.785,"2,013,454","2,651,605","$3,596,871",$1.79,$1.78 - $1.795,$1.7865,...,"1,415,925,204",Financial Services,ETHI / BFL / VGAD,$0.00,$0.0735,$0.00,NaN,NaN,NaN,MGF
6,Value,$2.02,0.00\n(0.00),$2.02\n-\n$2.03,"1,130,653","1,119,174","$2,290,677",$2.03,$2.02 - $2.03,$2.025,...,"882,914,478",Financial Services,LSF / WLE / PNI,$0.00,$0.172,$0.00,NaN,NaN,NaN,MXT
7,Value,$1.38,0.00\n(0.00),$1.38\n-\n$1.385,"968,339","758,688","$1,341,733",$1.39,$1.375 - $1.395,$1.3855,...,"1,260,001,474",Financial Services,PNI / VAF / LSF,$0.00,$0.13,$0.294,NaN,NaN,NaN,WLE
8,Value,$2.86,0.06\n(2.14%),$2.82\n-\n$2.86,"533,122","379,764","$1,509,543",$2.81,$2.80 - $2.86,$2.83,...,"619,457,738",Financial Services,MXT / WLE / PNI,$0.00,$0.105,$0.00,NaN,NaN,NaN,LSF
9,Value,$1.54,0.03\n(1.99%),$1.54\n-\n$1.56,"692,079","983,308","$1,060,390",$1.52,$1.52 - $1.56,$1.532,...,"1,107,014,423",Financial Services,VAF / PNI / WLE,$0.00,$0.155,$0.00,NaN,NaN,NaN,WAM


In [33]:
# Output 19th 10 tickers
tickers = all_tickers[180:190]  # Replace with the desired tickers
final_df19 = scrape_multiple_stock_data(tickers)
final_df19

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Ticker,Breakdown,Recommendation,Last Updated
0,Value,$2.82,0.02\n(0.71%),$2.82\n-\n$2.83,"203,553","260,725","$575,925",$2.83,$2.82 - $2.84,$2.83,...,"576,511,084",Financial Services,PSI / WAM / VAF,$0.00,$0.095,$0.00,MFF,NaN,NaN,NaN
1,Value,$1.705,0.00\n(0.00),$1.70\n-\n$1.71,"118,615","240,370","$202,279",$1.70,$1.70 - $1.71,$1.7055,...,"802,824,937",Financial Services,IFL / IXJ / MFG,$0.00,$0.082,$0.00,BKI,NaN,NaN,NaN
2,Value,$9.49,0.01\n(0.11%),$9.49\n-\n$9.53,"20,678","19,628","$196,547",$9.48,$9.48 - $9.53,$9.505,...,"126,481,315",Financial Services,QPON / LFG / HLI,$0.00,$0.37,$0.00,AUI,NaN,NaN,NaN
3,Value,$244.00,5.90\n(2.48%),$243.95\n-\n$244.10,"1,098,729","940,363","$269,328,077",$239.99,$239.60 - $244.155,$243.065,...,"483,064,837",Healthcare,COH / SHL / RMD,$4.53,$3.6292,-$1.366,CSL,17 Buy\n· 3 Hold\n· 0 Sell,Strong Buy,01/10/23
4,Value,$9.24,0.14\n(1.54%),$9.20\n-\n$9.27,"857,962","1,053,843","$7,981,502",$9.24,$9.18 - $9.445,$9.305,...,"323,364,555",Healthcare,ANN / SNZ / NEU,-$0.149,$0.00,$0.028,TLX,10 Buy\n· 1 Hold\n· 1 Sell,Strong Buy,01/10/23
5,Value,$12.25,0.96\n(8.50%),$12.08\n-\n$12.39,"1,682,266","443,894","$21,465,416",$12.41,$12.075 - $13.92,$12.745,...,"126,565,676",Healthcare,NAN / HLS / SNZ,$0.436,$0.00,$0.322,NEU,7 Buy\n· 1 Hold\n· 1 Sell,Buy,01/10/23
6,Value,$15.46,0.53\n(3.55%),$15.46\n-\n$15.59,"49,371","74,633","$762,357",$15.05,$15.05 - $15.63,$15.44,...,"49,410,338",Healthcare,REG / API / EHE,$0.591,$0.05,$3.328,CUV,6 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23
7,Value,$3.25,0.08\n(2.52%),$3.24\n-\n$3.28,"142,616","274,450","$471,058",$3.30,$3.25 - $3.35,$3.305,...,"59,640,755",Healthcare,BOT / ONT / CAJ,-$0.244,$0.00,$0.662,AVH,13 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23
8,Value,$0.30,0.00\n(0.00),$0.30\n-\n$0.31,"843,810","1,898,613","$261,252",$0.305,$0.30 - $0.3175,$0.3095,...,"1,188,834,559",Healthcare,IMU / AFP / JHC,-$0.045,$0.00,$0.107,IMM,8 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23
9,Value,$3.20,0.00\n(0.00),$3.09\n-\n$3.20,"2,500",569,"$8,000",$3.20,$3.20 - $3.20,$3.20,...,"104,866,260",Healthcare,IMU / MYX / IMM,$0.10,$0.0069,$0.264,AFP,5 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23


In [34]:
# Output 20th 10 tickers
tickers = all_tickers[190:200]  # Replace with the desired tickers
final_df20 = scrape_multiple_stock_data(tickers)
final_df20

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$0.37,0.005\n(1.37%),$0.37\n-\n$0.375,"1,608,612","1,159,621","$593,995",$0.37,$0.36 - $0.375,$0.3695,...,"814,204,825",Healthcare,CAU / AVR / CU6,-$0.111,$0.00,-$0.093,7 Buy\n· 2 Hold\n· 1 Sell,Buy,01/10/23,MSB
1,Value,$3.72,0.03\n(0.81%),$3.64\n-\n$3.82,"133,137","364,295","$499,472",$3.59,$3.59 - $3.82,$3.75,...,"85,074,288",Healthcare,AVR / MSB / CAU,-$3.86,$0.00,$0.201,0 Buy\n· 3 Hold\n· 5 Sell,Strong Sell,01/10/23,MYX
2,Value,$0.048,0.005\n(11.63%),$0.047\n-\n$0.048,"31,216,017","23,908,889","$1,450,452",$0.043,$0.043 - $0.049,$0.0465,...,"7,164,974,859",Healthcare,AFP / IMM / MYX,-$0.006,$0.00,$0.00,5 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,IMU
3,Value,$6.38,0.07\n(1.11%),$6.34\n-\n$6.50,"2,122,244","2,393,321","$13,584,835",$6.31,$6.30 - $6.52,$6.40,...,"689,632,247",Real Estate,CHC / SCP / NSR,-$0.34,$0.16,$7.801,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,LLC
4,Value,$11.54,0.01\n(-0.09%),$11.37\n-\n$11.60,"206,392","381,856","$2,377,807",$11.74,$11.36 - $11.74,$11.52,...,"177,350,573",Real Estate,AVN / BWP / CIP,-$0.123,$0.00,-$1.742,10 Buy\n· 1 Hold\n· 1 Sell,Strong Buy,01/10/23,PXA
5,Value,$16.80,0.26\n(1.57%),$16.00\n-\n$16.86,"132,386","227,304","$2,225,714",$16.79,$16.67 - $16.92,$16.81,...,"104,545,131",Real Estate,INA / CQR / CIP,$0.78,$0.115,$5.02,7 Buy\n· 4 Hold\n· 0 Sell,Buy,01/10/23,LIC
6,Value,$0.52,0.00\n(0.00),$0.51\n-\n$0.53,0,"10,468",$0,‐,‐ - ‐,‐,...,"521,881,164",Real Estate,SRV / APZ / APD,$0.051,$0.04,$1.078,NaN,NaN,NaN,UOS
7,Value,$2.43,0.00\n(0.00),‐\n-\n$2.42,0,187,$0,‐,‐ - ‐,‐,...,"296,613,736",Real Estate,COF / RFF / QAL,$0.21,$0.0387,$1.712,4 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,WTN
8,Value,$2.34,0.04\n(1.74%),$2.28\n-\n$2.35,"23,634","96,120","$54,606",$2.315,$2.27 - $2.35,$2.31,...,"296,016,053",Real Estate,RFF / COF / WTN,$0.076,$0.075,$1.209,6 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,QAL
9,Value,$1.18,0.00\n(0.00),$1.18\n-\n$1.20,0,"89,161",$0,‐,‐ - ‐,‐,...,"470,679,213",Real Estate,PWG / HPI / AQR,$0.139,$0.075,$0.099,NaN,NaN,NaN,PPC


In [35]:
# Output 21st 10 tickers
tickers = all_tickers[200:210]  # Replace with the desired tickers
final_df21 = scrape_multiple_stock_data(tickers)
final_df21

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$4.60,0.05\n(1.10%),$4.52\n-\n$4.60,"31,555","42,038","$145,351",$4.57,$4.55 - $4.66,$4.605,...,"82,418,418",Real Estate,REP / DXC / CDP,$0.38,$0.20,$0.733,4 Buy\n· 4 Hold\n· 0 Sell,Buy,01/10/23,CWP
1,Value,$2.90,0.03\n(-1.02%),$2.90\n-\n$3.01,"38,533","12,691","$111,983",$2.92,$2.89 - $2.93,$2.905,...,"96,920,388",Real Estate,UOS / APZ / APD,$0.306,$0.22,$0.947,7 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,SRV
2,Value,$0.65,0.005\n(0.78%),$0.645\n-\n$0.69,"171,941","20,661","$111,491",$0.65,$0.64 - $0.66,$0.6485,...,"272,123,142",Real Estate,NPR / AOF / WOT,$0.012,$0.00,$0.882,NaN,NaN,NaN,FRI
3,Value,$3.90,0.42\n(12.07%),$3.90\n-\n$3.98,"1,270,163","1,236,910","$4,953,904",$3.65,$3.65 - $4.02,$3.90,...,"187,854,564",Technology,FCL / NTO / IFM,-$0.222,$0.00,$0.451,NaN,NaN,NaN,WBT
4,Value,$0.125,0.01\n(8.70%),$0.125\n-\n$0.13,"6,817,292","5,315,701","$870,131",$0.115,$0.115 - $0.135,$0.1275,...,"1,747,143,252",Technology,QOR / EPD / DUG,-$0.004,$0.00,$0.003,NaN,NaN,NaN,4DS
5,Value,$0.41,0.01\n(2.50%),$0.40\n-\n$0.41,"81,947","163,552","$32,936",$0.40,$0.40 - $0.41,$0.402,...,"254,847,013",Technology,CRO / PX1 / LME,-$0.036,$0.00,$0.101,NaN,NaN,NaN,AXE
6,Value,$0.061,0.003\n(-4.69%),$0.06\n-\n$0.061,"3,486,425","2,227,246","$217,614",$0.064,$0.059 - $0.065,$0.0625,...,"1,528,045,654",Technology,VLT / PTG / WCG,-$0.009,$0.00,$0.009,NaN,NaN,NaN,BLG
7,Value,$0.17,0.005\n(3.03%),$0.155\n-\n$0.17,"12,400","21,843","$2,016",$0.15,$0.15 - $0.17,$0.1625,...,"105,916,654",Technology,CF1 / FLX / VIG,-$0.068,$0.00,$0.057,NaN,NaN,NaN,RVS
8,Value,$0.003,0.001\n(-25.00%),$0.003\n-\n$0.004,"6,371,418","291,437","$21,253",$0.004,$0.003 - $0.004,$0.0035,...,"768,379,278",Technology,DC2 / SIS / AO1,-$0.031,$0.00,$0.014,NaN,NaN,NaN,PVS
9,Value,$60.05,0.07\n(0.12%),$59.88\n-\n$61.00,"453,986","553,763","$27,360,969",$60.61,$59.79 - $60.96,$60.205,...,"333,315,290",Technology,APT / XRO / NXT,$0.693,$0.15,-$0.893,10 Buy\n· 8 Hold\n· 1 Sell,Buy,01/10/23,WTC


In [36]:
# Output 22nd 10 tickers
tickers = all_tickers[210:220]  # Replace with the desired tickers
final_df22 = scrape_multiple_stock_data(tickers)
final_df22

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$112.29,1.54\n(1.39%),$111.95\n-\n$112.60,"326,421","381,557","$36,711,769",$113.50,$110.37 - $113.67,$112.515,...,"151,468,059",Technology,APT / WTC / NXT,-$0.711,$0.00,$0.556,15 Buy\n· 4 Hold\n· 2 Sell,Buy,01/10/23,XRO
1,Value,$12.45,0.21\n(1.72%),$12.40\n-\n$12.57,"1,000,437","1,105,434","$12,473,859",$12.43,$12.34 - $12.55,$12.47,...,"514,707,798",Technology,ALU / TNE / DDR,-$0.009,$0.00,$4.315,18 Buy\n· 1 Hold\n· 1 Sell,Strong Buy,01/10/23,NXT
2,Value,$40.81,0.12\n(-0.29%),$40.70\n-\n$41.48,"587,422","305,331","$24,100,381",$41.50,$40.67 - $41.55,$41.03,...,"131,926,838",Technology,TNE / NXT / DDR,$0.499,$0.54,$0.091,9 Buy\n· 8 Hold\n· 3 Sell,Hold,01/10/23,ALU
3,Value,$15.23,0.07\n(0.46%),$15.15\n-\n$15.50,"557,360","582,802","$8,495,926",$15.34,$15.11 - $15.41,$15.245,...,"324,674,728",Technology,ALU / NXT / DDR,$0.298,$0.1744,$0.162,8 Buy\n· 7 Hold\n· 2 Sell,Buy,01/10/23,TNE
4,Value,$10.15,0.22\n(2.22%),$10.09\n-\n$10.20,"646,712","1,078,817","$6,570,826",$10.10,$10.04 - $10.35,$10.16,...,"159,063,214",Technology,360 / MAQ / CDA,-$0.268,$0.00,$0.461,11 Buy\n· 5 Hold\n· 1 Sell,Buy,01/10/23,MP1
5,Value,$7.81,0.00\n(0.00),$7.73\n-\n$7.86,"436,263","517,440","$3,451,791",$7.92,$7.69 - $8.07,$7.91,...,"202,668,276",Technology,MP1 / MAQ / CDA,-$0.277,$0.00,$0.29,9 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,360
6,Value,$63.66,0.46\n(0.73%),$63.19\n-\n$65.36,"15,035","10,701","$966,170",$63.45,$63.35 - $65.08,$64.26,...,"24,344,268",Technology,360 / MP1 / CDA,$0.816,$0.00,$12.358,7 Buy\n· 1 Hold\n· 2 Sell,Buy,01/10/23,MAQ
7,Value,$4.32,0.07\n(1.65%),$4.30\n-\n$4.34,"341,078","674,017","$1,486,124",$4.47,$4.30 - $4.50,$4.355,...,"275,066,106",Technology,ISX / SP1 / DTL,-$0.19,$0.00,$0.102,14 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,SDR
8,Value,$7.23,0.08\n(1.12%),$7.10\n-\n$7.24,"640,561","313,561","$4,620,996",$7.20,$7.11 - $7.29,$7.215,...,"154,704,064",Technology,AD8 / ISX / SP1,$0.239,$0.219,$0.343,7 Buy\n· 3 Hold\n· 2 Sell,Buy,01/10/23,DTL
9,Value,$11.06,0.14\n(1.28%),$11.05\n-\n$11.08,"349,809","161,092","$3,851,619",$11.00,$10.95 - $11.14,$11.01,...,"180,243,333",Technology,MP1 / 360 / MAQ,$0.428,$0.355,$1.277,7 Buy\n· 3 Hold\n· 0 Sell,Strong Buy,01/10/23,DDR


In [37]:
# Output 23rd 10 tickers
tickers = all_tickers[220:230]  # Replace with the desired tickers
final_df23 = scrape_multiple_stock_data(tickers)
final_df23

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$8.00,0.01\n(-0.13%),$7.85\n-\n$8.14,"219,800","309,693","$1,766,131",$8.09,$7.96 - $8.15,$8.035,...,"181,316,113",Technology,MAQ / 360 / MP1,$0.374,$0.185,$0.73,8 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,CDA
1,Value,$13.22,0.42\n(-3.08%),$13.22\n-\n$13.49,"78,352","159,319","$1,045,137",$13.51,$13.20 - $13.63,$13.34,...,"83,108,836",Technology,DTL / HSN / NEA,-$0.035,$0.00,$0.671,8 Buy\n· 2 Hold\n· 1 Sell,Buy,01/10/23,AD8
2,Value,$0.735,0.015\n(2.08%),$0.735\n-\n$0.74,"3,215,285","3,499,647","$2,404,135",$0.75,$0.735 - $0.76,$0.7475,...,"488,610,194",Technology,OTW / RUL / BVS,-$0.15,$0.00,$0.439,0 Buy\n· 3 Hold\n· 4 Sell,Strong Sell,01/10/23,NVX
3,Value,$0.355,0.015\n(4.41%),$0.34\n-\n$0.355,"54,793","65,668","$18,988",$0.35,$0.34 - $0.355,$0.3465,...,"350,353,001",Technology,EOL / PYG / MSL,$0.002,$0.00,$0.022,5 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,SPZ
4,Value,$0.60,0.02\n(3.45%),$0.585\n-\n$0.61,"70,750","152,866","$42,718",$0.60,$0.59 - $0.62,$0.604,...,"184,631,174",Technology,LME / AXE / CRO,-$0.023,$0.00,$0.05,4 Buy\n· 2 Hold\n· 0 Sell,Buy,01/10/23,ERD
5,Value,$0.58,0.00\n(0.00),$0.575\n-\n$0.595,0,"3,478",$0,‐,‐ - ‐,‐,...,"171,885,104",Technology,PX1 / ACE / CRO,-$0.064,$0.00,$0.362,NaN,NaN,NaN,RWL
6,Value,$0.17,0.00\n(0.00),$0.16\n-\n$0.175,"293,687","334,390","$49,927",$0.17,$0.17 - $0.17,$0.17,...,"510,551,547",Technology,EIQ / IHR / DTS,-$0.014,$0.00,$0.00,NaN,NaN,NaN,DTZ
7,Value,$0.50,0.00\n(0.00),$0.50\n-\n$0.55,"3,572","17,599","$1,786",$0.50,$0.50 - $0.50,$0.50,...,"160,024,338",Technology,FCT / NNT / IHR,-$0.05,$0.00,$0.13,4 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,IKE
8,Value,$0.55,0.02\n(3.77%),$0.53\n-\n$0.56,"10,112","67,243","$5,521",$0.53,$0.53 - $0.55,$0.546,...,"134,179,626",Technology,CCR / DUB / EVS,-$0.034,$0.00,$0.063,NaN,NaN,NaN,SMN
9,Value,$3.87,0.02\n(0.52%),$3.86\n-\n$3.87,"18,047,148","20,520,799","$69,878,775",$3.88,$3.855 - $3.89,$3.87,...,"11,554,427,353",Communication Services,REA / CAR / TPG,$0.167,$0.17,$0.382,14 Buy\n· 2 Hold\n· 1 Sell,Strong Buy,01/10/23,TLS


In [38]:
# Output 24th 10 tickers
tickers = all_tickers[230:240]  # Replace with the desired tickers
final_df24 = scrape_multiple_stock_data(tickers)
final_df24

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$5.29,0.04\n(0.76%),$5.26\n-\n$5.33,"11,033,733","1,355,889","$58,446,829",$5.33,$5.28 - $5.35,$5.30,...,"1,859,341,669",Communication Services,CAR / SPK / SEK,$0.212,$0.18,-$0.41,6 Buy\n· 7 Hold\n· 1 Sell,Buy,01/10/23,TPG
1,Value,$4.62,0.01\n(-0.22%),$4.59\n-\n$4.63,"610,921","835,374","$2,822,108",$4.65,$4.59 - $4.66,$4.62,...,"1,830,097,423",Communication Services,SEK / TPG / CAR,$0.606,$0.2493,$0.615,5 Buy\n· 6 Hold\n· 1 Sell,Buy,01/10/23,SPK
2,Value,$6.84,0.04\n(0.59%),$6.46\n-\n$6.95,"379,717","332,170","$2,604,601",$6.85,$6.81 - $6.93,$6.86,...,"433,887,294",Communication Services,NEC / VOC / UWL,$0.045,$0.334,$1.941,0 Buy\n· 4 Hold\n· 7 Sell,Strong Sell,01/10/23,CNU
3,Value,$1.98,0.07\n(-3.42%),$1.98\n-\n$2.07,"90,363","175,038","$183,114",$2.04,$1.98 - $2.05,$2.025,...,"464,959,921",Communication Services,ABB / NWS / OML,-$0.045,$0.00,$0.67,4 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,TUA
4,Value,$3.58,0.34\n(-8.67%),$3.58\n-\n$3.60,"1,858,790","687,914","$6,700,499",$3.58,$3.51 - $3.69,$3.605,...,"239,209,360",Communication Services,TUA / OML / OPC,$0.091,$0.00,-$0.444,4 Buy\n· 5 Hold\n· 0 Sell,Buy,01/10/23,ABB
5,Value,$0.03,0.001\n(-3.23%),$0.031\n-\n$0.037,700,"8,811",$21,$0.03,$0.03 - $0.03,$0.03,...,"13,572,508,577",Communication Services,SWM / SKT / HT1,-$0.034,$0.00,$0.025,NaN,NaN,NaN,HTA
6,Value,$0.64,0.005\n(0.79%),$0.64\n-\n$0.65,"167,114","309,479","$106,242",$0.64,$0.6175 - $0.645,$0.6355,...,"491,174,540",Communication Services,HT1 / IGL / SKT,-$0.129,$0.00,$0.084,8 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,SLC
7,Value,$0.20,0.01\n(5.26%),$0.19\n-\n$0.20,"14,073","34,196","$2,746",$0.19,$0.19 - $0.20,$0.195,...,"208,208,937",Communication Services,UNL / ICI / ICU,-$0.031,$0.00,$0.032,NaN,NaN,NaN,SWP
8,Value,$0.043,0.006\n(-12.25%),$0.04\n-\n$0.05,"5,658,644","132,578","$246,508",$0.045,$0.043 - $0.045,$0.0435,...,"763,741,605",Communication Services,IOU / PPL / RMY,-$0.003,$0.00,$0.033,NaN,NaN,NaN,FSG
9,Value,$12.36,0.30\n(2.49%),$12.32\n-\n$12.41,"4,393,717","5,209,518","$54,201,487",$12.23,$12.20 - $12.40,$12.335,...,"3,088,257,050",Industrials,SYD / BXB / CPU,$0.021,$0.58,$1.075,9 Buy\n· 7 Hold\n· 2 Sell,Buy,01/10/23,TCL


In [39]:
# Output 25th 10 tickers
tickers = all_tickers[240:250]  # Replace with the desired tickers
final_df25 = scrape_multiple_stock_data(tickers)
final_df25

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$7.11,0.10\n(1.43%),$7.03\n-\n$7.18,"577,985","478,754","$4,108,958",$7.11,$7.08 - $7.15,$7.11,...,"1,472,551,137",Industrials,SVW / REH / QAN,$0.029,$0.0372,$5.688,2 Buy\n· 7 Hold\n· 7 Sell,Sell,01/10/23,AIA
1,Value,$5.18,0.10\n(1.97%),$5.18\n-\n$5.19,"8,557,318","10,408,004","$44,356,215",$5.16,$5.16 - $5.21,$5.185,...,"1,724,454,680",Industrials,WOR / ALX / IFT,$0.93,$0.00,-$0.395,15 Buy\n· 3 Hold\n· 1 Sell,Strong Buy,01/10/23,QAN
2,Value,$5.64,0.24\n(4.44%),$5.49\n-\n$5.80,"2,465,904","2,171,953","$13,822,436",$5.48,$5.45 - $5.67,$5.605,...,"1,450,833,707",Industrials,IFT / WOR / QAN,$0.227,$0.40,$2.97,5 Buy\n· 6 Hold\n· 1 Sell,Buy,01/10/23,ALX
3,Value,$3.54,0.09\n(2.61%),$3.52\n-\n$3.56,"3,728,592","5,294,734","$13,161,616",$3.50,$3.47 - $3.54,$3.53,...,"1,840,703,982",Industrials,CIM / CWY / ALQ,$0.176,$0.15,$2.245,8 Buy\n· 8 Hold\n· 2 Sell,Buy,01/10/23,AZJ
4,Value,$2.90,0.12\n(4.32%),$2.89\n-\n$2.94,"5,153,651","2,900,798","$14,913,841",$2.83,$2.81 - $2.92,$2.895,...,"1,766,712,012",Industrials,ALQ / CWY / AZJ,$0.093,$0.081,$1.207,8 Buy\n· 9 Hold\n· 0 Sell,Buy,01/10/23,QUB
5,Value,$0.62,0.005\n(0.81%),$0.62\n-\n$0.63,"1,145,554","533,840","$710,166",$0.62,$0.615 - $0.63,$0.62,...,"3,368,464,315",Industrials,BIN / APM / VNT,-$0.197,$0.055,$0.457,1 Buy\n· 6 Hold\n· 5 Sell,Sell,01/10/23,AIZ
6,Value,$6.27,0.09\n(1.46%),$6.25\n-\n$6.32,"269,219","411,016","$1,689,070",$6.25,$6.18 - $6.32,$6.275,...,"269,358,541",Industrials,IPH / JLG / SLK,$0.09,$0.17,-$0.244,10 Buy\n· 6 Hold\n· 0 Sell,Buy,01/10/23,KLS
7,Value,$2.72,0.00\n(0.00),$2.72\n-\n$2.73,"108,896","241,681","$297,008",$2.72,$2.72 - $2.74,$2.725,...,"495,761,667",Industrials,MAD / MND / RDX,$0.197,$0.201,-$3.973,5 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,DBI
8,Value,$7.07,0.00\n(0.00),$6.60\n-\n‐,0,145,$0,‐,‐ - ‐,‐,...,"178,622,819",N/A,-,$0.00,$0.00,$0.00,5 Buy\n· 4 Hold\n· 0 Sell,Buy,01/10/23,FRW
9,Value,$8.38,0.09\n(-1.06%),$8.37\n-\n$8.41,"81,154,643","10,227,333","$681,669,864",$8.35,$8.30 - $8.43,$8.395,...,"1,722,747,671",Utilities,APA / AST / MCY,$0.609,$0.365,$0.65,5 Buy\n· 4 Hold\n· 0 Sell,Buy,01/10/23,ORG


In [40]:
# Output 26th 10 tickers
tickers = all_tickers[250:260]  # Replace with the desired tickers
final_df26 = scrape_multiple_stock_data(tickers)
final_df26

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$4.48,0.01\n(0.22%),$4.47\n-\n$4.65,"5,105","42,508","$23,449",$4.60,$4.48 - $4.65,$4.595,...,"1,266,374,569",Utilities,SKI / AGL / MCY,$0.197,$0.1601,$2.286,2 Buy\n· 3 Hold\n· 6 Sell,Sell,01/10/23,MEZ
1,Value,$8.37,0.15\n(1.83%),$8.34\n-\n$8.38,"1,685,501","2,781,470","$14,094,994",$8.39,$8.30 - $8.40,$8.36,...,"1,282,957,939",Utilities,AST / MCY / AGL,$0.223,$0.55,$0.457,7 Buy\n· 7 Hold\n· 0 Sell,Buy,01/10/23,APA
2,Value,$10.59,0.07\n(0.67%),$10.57\n-\n$10.62,"1,574,975","1,721,986","$16,690,721",$10.62,$10.58 - $10.665,$10.605,...,"672,747,233",Utilities,MCY / MEZ / SKI,-$1.879,$0.31,$2.882,9 Buy\n· 5 Hold\n· 0 Sell,Buy,01/10/23,AGL
3,Value,$5.60,0.12\n(2.19%),$5.30\n-\n$5.60,874,"6,789","$4,764",$5.50,$5.40 - $5.60,$5.45,...,"1,391,227,239",Utilities,AGL / MEZ / AST,$0.074,$0.2011,$3.386,1 Buy\n· 5 Hold\n· 5 Sell,Sell,01/10/23,MCY
4,Value,$7.29,0.04\n(0.55%),$7.08\n-\n$7.70,"3,602","9,956","$26,207",$7.27,$7.25 - $7.30,$7.275,...,"254,546,234",Utilities,GNE / IFN / TLT,$0.163,$0.3167,$3.014,7 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,CEN
5,Value,$2.17,0.02\n(0.93%),$2.11\n-\n$2.20,"18,082","20,899","$39,339",$2.15,$2.15 - $2.20,$2.175,...,"1,074,640,281",Utilities,CEN / TLT / IFN,$0.185,$0.1642,$1.966,5 Buy\n· 3 Hold\n· 1 Sell,Buy,01/10/23,GNE
6,Value,$1.635,0.015\n(0.93%),$1.635\n-\n$1.67,"38,498","79,426","$63,254",$1.64,$1.635 - $1.67,$1.643,...,"152,720,578",Utilities,GNX / LGI / EWC,$0.00,$0.069,$0.00,NaN,NaN,NaN,D2O
7,Value,$0.16,0.01\n(6.67%),$0.155\n-\n$0.16,"1,750,005","372,851","$272,608",$0.155,$0.1525 - $0.16,$0.156,...,"1,385,177,140",Utilities,D2O / LGI / EWC,-$0.001,$0.00,$0.143,5 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,GNX
8,Value,$2.07,0.07\n(3.50%),$1.95\n-\n$2.07,"10,829","22,197","$22,226",$2.06,$2.05 - $2.07,$2.05,...,"88,293,771",Utilities,GNX / D2O / EWC,$0.073,$0.0231,$0.52,0 Buy\n· 4 Hold\n· 4 Sell,Sell,01/10/23,LGI


## Merge the dataframes

In [41]:
# Append df2 to df1
dfs = [final_df1, final_df2, final_df3, final_df4, final_df5, final_df6, final_df7, final_df8, final_df9, final_df10,
       final_df11, final_df12, final_df13, final_df14, final_df15, final_df16, final_df17, final_df18, final_df19, final_df20,
       final_df21, final_df22, final_df23, final_df24, final_df25, final_df26]
result_df = pd.concat(dfs, ignore_index=True)

# Print the resulting DataFrame
result_df

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$31.26,1.02\n(3.37%),$29.88\n-\n$31.35,"160,245","245,483","$4,999,714",$31.07,$30.56 - $31.45,$31.20,...,"82,220,441",Consumer Cyclical,CTD / WEB / SUL,$1.079,$0.62,$0.60,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,ARB
1,Value,$11.08,0.17\n(1.56%),$10.77\n-\n$11.35,"366,353","441,797","$4,045,735",$10.95,$10.725 - $11.10,$11.045,...,"140,894,696",Consumer Cyclical,SGR / LNW / SKC,$0.693,$0.39,$0.526,11 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,GUD
2,Value,$10.12,0.07\n(0.70%),$9.96\n-\n$10.92,"37,398","127,350","$378,557",$10.16,$10.06 - $10.22,$10.12,...,"100,484,131",Consumer Cyclical,BGP / CTT / AX1,$0.217,$0.125,$0.722,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,PWH
3,Value,$16.10,0.65\n(-3.88%),$16.15\n-\n$17.10,"5,570",758,"$90,352",$16.75,$16.01 - $16.75,$16.22,...,"13,569,557",Consumer Cyclical,NDO / ALG / BST,$1.811,$0.90,$3.324,NaN,NaN,NaN,SFC
4,Value,$0.89,0.00\n(0.00),$0.845\n-\n$0.90,0,"11,146",$0,‐,‐ - ‐,‐,...,"81,285,172",Consumer Cyclical,ISU / KED / CCX,$0.084,$0.0233,$0.986,NaN,NaN,NaN,AHL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Value,$7.29,0.04\n(0.55%),$7.08\n-\n$7.70,"3,602","9,956","$26,207",$7.27,$7.25 - $7.30,$7.275,...,"254,546,234",Utilities,GNE / IFN / TLT,$0.163,$0.3167,$3.014,7 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,CEN
255,Value,$2.17,0.02\n(0.93%),$2.11\n-\n$2.20,"18,082","20,899","$39,339",$2.15,$2.15 - $2.20,$2.175,...,"1,074,640,281",Utilities,CEN / TLT / IFN,$0.185,$0.1642,$1.966,5 Buy\n· 3 Hold\n· 1 Sell,Buy,01/10/23,GNE
256,Value,$1.635,0.015\n(0.93%),$1.635\n-\n$1.67,"38,498","79,426","$63,254",$1.64,$1.635 - $1.67,$1.643,...,"152,720,578",Utilities,GNX / LGI / EWC,$0.00,$0.069,$0.00,NaN,NaN,NaN,D2O
257,Value,$0.16,0.01\n(6.67%),$0.155\n-\n$0.16,"1,750,005","372,851","$272,608",$0.155,$0.1525 - $0.16,$0.156,...,"1,385,177,140",Utilities,D2O / LGI / EWC,-$0.001,$0.00,$0.143,5 Buy\n· 1 Hold\n· 0 Sell,Strong Buy,01/10/23,GNX


In [43]:
## Inspect the datagrame
print(result_df.dtypes)
print(result_df.describe())
result_df.head(10)

Attribute
index                                                                                   object
Last (Price)                                                                            object
Change                                                                                  object
Bid / Ask                                                                               object
Volume                                                                                  object
Volume (4w avg)                                                                         object
Turnover                                                                                object
Open                                                                                    object
Day Range                                                                               object
VWAP                                                                                    object
Prev Close                              

Attribute,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$31.26,1.02\n(3.37%),$29.88\n-\n$31.35,"160,245","245,483","$4,999,714",$31.07,$30.56 - $31.45,$31.20,...,"82,220,441",Consumer Cyclical,CTD / WEB / SUL,$1.079,$0.62,$0.60,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,ARB
1,Value,$11.08,0.17\n(1.56%),$10.77\n-\n$11.35,"366,353","441,797","$4,045,735",$10.95,$10.725 - $11.10,$11.045,...,"140,894,696",Consumer Cyclical,SGR / LNW / SKC,$0.693,$0.39,$0.526,11 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,GUD
2,Value,$10.12,0.07\n(0.70%),$9.96\n-\n$10.92,"37,398","127,350","$378,557",$10.16,$10.06 - $10.22,$10.12,...,"100,484,131",Consumer Cyclical,BGP / CTT / AX1,$0.217,$0.125,$0.722,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,PWH
3,Value,$16.10,0.65\n(-3.88%),$16.15\n-\n$17.10,"5,570",758,"$90,352",$16.75,$16.01 - $16.75,$16.22,...,"13,569,557",Consumer Cyclical,NDO / ALG / BST,$1.811,$0.90,$3.324,NaN,NaN,NaN,SFC
4,Value,$0.89,0.00\n(0.00),$0.845\n-\n$0.90,0,"11,146",$0,‐,‐ - ‐,‐,...,"81,285,172",Consumer Cyclical,ISU / KED / CCX,$0.084,$0.0233,$0.986,NaN,NaN,NaN,AHL
5,Value,$0.155,0.005\n(3.33%),$0.145\n-\n$0.155,"82,262","432,421","$12,513",$0.155,$0.145 - $0.155,$0.152,...,"290,240,290",Consumer Cyclical,DNA / BBT / MFD,$0.025,$0.00,$0.202,NaN,NaN,NaN,VMT
6,Value,$0.081,0.0005\n(0.62%),$0.078\n-\n$0.085,"92,000","26,007","$7,490",$0.081,$0.081 - $0.082,$0.0815,...,"195,448,061",Consumer Cyclical,XRG / ABV / CLU,$0.01,$0.00,$0.031,NaN,NaN,NaN,RPM
7,Value,$0.044,0.002\n(4.76%),$0.043\n-\n$0.046,"231,792","154,298","$10,160",$0.042,$0.042 - $0.045,$0.044,...,"379,466,944",Consumer Cyclical,XRG / RPM / UCW,$0.004,$0.00,$0.017,NaN,NaN,NaN,ABV
8,Value,$0.004,0.00\n(0.00),$0.004\n-\n$0.005,"286,645","1,241,239","$1,147",$0.004,$0.004 - $0.004,$0.004,...,"1,210,952,783",Consumer Cyclical,MHI / SIX / AUA,$0.00,$0.00,$0.008,NaN,NaN,NaN,DDT
9,Value,$0.014,0.00\n(0.00),$0.012\n-\n$0.014,176,"555,355",$2,$0.014,$0.014 - $0.014,$0.014,...,"338,998,012",Consumer Cyclical,AUA / MHI / DDT,-$0.02,$0.00,$0.003,NaN,NaN,NaN,SIX


In [46]:
# Rename a specific column in the DataFrame
result_df = result_df.rename(columns={'Market Cap\n\n\n\n\n\n\n\n\n\nExcludes mandatory escrowed shares and foreign shares.': 'Market Cap'})
print(result_df.head())
print(result_df.columns)

Attribute  index Last (Price)          Change          Bid / Ask   Volume  \
0          Value       $31.26   1.02\n(3.37%)  $29.88\n-\n$31.35  160,245   
1          Value       $11.08   0.17\n(1.56%)  $10.77\n-\n$11.35  366,353   
2          Value       $10.12   0.07\n(0.70%)   $9.96\n-\n$10.92   37,398   
3          Value       $16.10  0.65\n(-3.88%)  $16.15\n-\n$17.10    5,570   
4          Value        $0.89    0.00\n(0.00)   $0.845\n-\n$0.90        0   

Attribute Volume (4w avg)    Turnover    Open         Day Range     VWAP  ...  \
0                 245,483  $4,999,714  $31.07   $30.56 - $31.45   $31.20  ...   
1                 441,797  $4,045,735  $10.95  $10.725 - $11.10  $11.045  ...   
2                 127,350    $378,557  $10.16   $10.06 - $10.22   $10.12  ...   
3                     758     $90,352  $16.75   $16.01 - $16.75   $16.22  ...   
4                  11,146          $0       ‐             ‐ - ‐        ‐  ...   

Attribute Shares Issued             Sector Similar

In [47]:
# Export the DataFrame to a CSV file
result_df.to_csv('Resources/06_ASX_Fundamental_Final.csv', index=False)

In [5]:
# Use the read_csv function to read the CSV file again into
fundamental_df_raw = pd.read_csv("Resources/06_ASX_Fundamental_Final.csv")

fundamental_df_raw.head(5)

,index,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Turnover,Open,Day Range,VWAP,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,Value,$31.26,1.02\n(3.37%),$29.88\n-\n$31.35,"160,245","245,483","$4,999,714",$31.07,$30.56 - $31.45,$31.20,...,"82,220,441",Consumer Cyclical,CTD / WEB / SUL,$1.079,$0.62,$0.60,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,ARB
1,Value,$11.08,0.17\n(1.56%),$10.77\n-\n$11.35,"366,353","441,797","$4,045,735",$10.95,$10.725 - $11.10,$11.045,...,"140,894,696",Consumer Cyclical,SGR / LNW / SKC,$0.693,$0.39,$0.526,11 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,GUD
2,Value,$10.12,0.07\n(0.70%),$9.96\n-\n$10.92,"37,398","127,350","$378,557",$10.16,$10.06 - $10.22,$10.12,...,"100,484,131",Consumer Cyclical,BGP / CTT / AX1,$0.217,$0.125,$0.722,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,PWH
3,Value,$16.10,0.65\n(-3.88%),$16.15\n-\n$17.10,"5,570",758,"$90,352",$16.75,$16.01 - $16.75,$16.22,...,"13,569,557",Consumer Cyclical,NDO / ALG / BST,$1.811,$0.90,$3.324,NaN,NaN,NaN,SFC
4,Value,$0.89,0.00\n(0.00),$0.845\n-\n$0.90,0,"11,146",$0,‐,‐ - ‐,‐,...,"81,285,172",Consumer Cyclical,ISU / KED / CCX,$0.084,$0.0233,$0.986,NaN,NaN,NaN,AHL


In [6]:
print(fundamental_df_raw.columns)

Index(['index', 'Last (Price)', 'Change', 'Bid / Ask', 'Volume',
       'Volume (4w avg)', 'Turnover', 'Open', 'Day Range', 'VWAP',
       'Prev Close', 'Last Trade', '1 Week', '1 Month', '2023 YTD', '1 Year',
       'vs Sector (1yr)', 'vs ASX 200 (1yr)', 'Market Cap', 'ASX Rank',
       'Sector Rank', 'Shares Issued', 'Sector', 'Similar Companies', 'EPS',
       'DPS', 'Book Value Per Share', 'Breakdown', 'Recommendation',
       'Last Updated', 'Ticker'],
      dtype='object')


In [7]:
# Drop some columns
columns_to_drop = ['index', 'Turnover', 'VWAP']
fundamental_df_raw = fundamental_df_raw.drop(columns=columns_to_drop)
fundamental_df_raw.head(5)

,Last (Price),Change,Bid / Ask,Volume,Volume (4w avg),Open,Day Range,Prev Close,Last Trade,1 Week,...,Shares Issued,Sector,Similar Companies,EPS,DPS,Book Value Per Share,Breakdown,Recommendation,Last Updated,Ticker
0,$31.26,1.02\n(3.37%),$29.88\n-\n$31.35,"160,245","245,483",$31.07,$30.56 - $31.45,$30.24,6:58pm 03/11 (AEDT),+3.30%,...,"82,220,441",Consumer Cyclical,CTD / WEB / SUL,$1.079,$0.62,$0.60,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,ARB
1,$11.08,0.17\n(1.56%),$10.77\n-\n$11.35,"366,353","441,797",$10.95,$10.725 - $11.10,$10.91,6:58pm 03/11 (AEDT),+2.59%,...,"140,894,696",Consumer Cyclical,SGR / LNW / SKC,$0.693,$0.39,$0.526,11 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,GUD
2,$10.12,0.07\n(0.70%),$9.96\n-\n$10.92,"37,398","127,350",$10.16,$10.06 - $10.22,$10.05,6:58pm 03/11 (AEDT),-0.88%,...,"100,484,131",Consumer Cyclical,BGP / CTT / AX1,$0.217,$0.125,$0.722,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,PWH
3,$16.10,0.65\n(-3.88%),$16.15\n-\n$17.10,"5,570",758,$16.75,$16.01 - $16.75,$16.75,6:58pm 03/11 (AEDT),-3.88%,...,"13,569,557",Consumer Cyclical,NDO / ALG / BST,$1.811,$0.90,$3.324,NaN,NaN,NaN,SFC
4,$0.89,0.00\n(0.00),$0.845\n-\n$0.90,0,"11,146",‐,‐ - ‐,$0.89,6:58pm 03/11 (AEDT),+3.49%,...,"81,285,172",Consumer Cyclical,ISU / KED / CCX,$0.084,$0.0233,$0.986,NaN,NaN,NaN,AHL


In [8]:
# Define a dictionary to map old column names to new column names
column_to_rename = {
    'Last (Price)': 'lastPrice',
    'Bid / Ask': 'Bid_Ask',
    'Volume': 'volumePerDay',
    'Volume (4w avg)': 'volume4wAvg',
    'Day Range': 'dayRange',
    'Prev Close': 'prevClose',
    'Last Trade': 'lastTrade',
    '1 Week': 'oneWeek',
    '1 Month': 'oneMonth',
    '2023 YTD': 'YTD2023',
    '1 Year': 'oneYear',
    'vs Sector (1yr)': 'vsSectorOneYr',
    'vs ASX 200 (1yr)': 'vsASX200OneYr',
    'Market Cap': 'marketCap',
    'ASX Rank': 'ASXRank',
    'Sector Rank': 'sectorRank',
    'Shares Issued': 'sharesIssued',
    'Similar Companies': 'similarCompanies',
    'Book Value Per Share': 'bookValuePerShare',
    'Last Updated': 'lastUpdated'
}

# Use the rename method to change column names
fundamental_df_raw = fundamental_df_raw.rename(columns=column_to_rename)
fundamental_df_raw.head(5)

,lastPrice,Change,Bid_Ask,volumePerDay,volume4wAvg,Open,dayRange,prevClose,lastTrade,oneWeek,...,sharesIssued,Sector,similarCompanies,EPS,DPS,bookValuePerShare,Breakdown,Recommendation,lastUpdated,Ticker
0,$31.26,1.02\n(3.37%),$29.88\n-\n$31.35,"160,245","245,483",$31.07,$30.56 - $31.45,$30.24,6:58pm 03/11 (AEDT),+3.30%,...,"82,220,441",Consumer Cyclical,CTD / WEB / SUL,$1.079,$0.62,$0.60,6 Buy\n· 5 Hold\n· 2 Sell,Hold,01/10/23,ARB
1,$11.08,0.17\n(1.56%),$10.77\n-\n$11.35,"366,353","441,797",$10.95,$10.725 - $11.10,$10.91,6:58pm 03/11 (AEDT),+2.59%,...,"140,894,696",Consumer Cyclical,SGR / LNW / SKC,$0.693,$0.39,$0.526,11 Buy\n· 2 Hold\n· 0 Sell,Strong Buy,01/10/23,GUD
2,$10.12,0.07\n(0.70%),$9.96\n-\n$10.92,"37,398","127,350",$10.16,$10.06 - $10.22,$10.05,6:58pm 03/11 (AEDT),-0.88%,...,"100,484,131",Consumer Cyclical,BGP / CTT / AX1,$0.217,$0.125,$0.722,7 Buy\n· 4 Hold\n· 1 Sell,Buy,01/10/23,PWH
3,$16.10,0.65\n(-3.88%),$16.15\n-\n$17.10,"5,570",758,$16.75,$16.01 - $16.75,$16.75,6:58pm 03/11 (AEDT),-3.88%,...,"13,569,557",Consumer Cyclical,NDO / ALG / BST,$1.811,$0.90,$3.324,NaN,NaN,NaN,SFC
4,$0.89,0.00\n(0.00),$0.845\n-\n$0.90,0,"11,146",‐,‐ - ‐,$0.89,6:58pm 03/11 (AEDT),+3.49%,...,"81,285,172",Consumer Cyclical,ISU / KED / CCX,$0.084,$0.0233,$0.986,NaN,NaN,NaN,AHL


In [9]:
# Iterate through all columns and strip the cells
for column in fundamental_df_raw.columns:
    fundamental_df_raw[column] = fundamental_df_raw[column].str.strip()
    
# Replace empty cells with NaN in the entire DataFrame
fundamental_df_raw.replace('', np.nan, inplace=True)
    
# Replace # replace all '' with None
fundamental_df_raw.replace('', None, inplace=True)   

# Replace some of the characters inside each column
fundamental_df_raw['lastPrice'] = fundamental_df_raw['lastPrice'].str.replace('$', '')    # Drop the dollar sign from "lastPrice"
fundamental_df_raw['Change'] = fundamental_df_raw['Change'].str.replace('\n', ' ')    # Remove newline characters from the "Change" column
fundamental_df_raw['Bid_Ask'] = fundamental_df_raw['Bid_Ask'].str.replace('\n', ' ')    # Remove newline characters from the "Bid_Ask" column
fundamental_df_raw['volumePerDay'] = fundamental_df_raw['volumePerDay'].str.replace(',', '')    # Remove newline characters from the "volumePerDay" column
fundamental_df_raw['volume4wAvg'] = fundamental_df_raw['volume4wAvg'].str.replace(',', '')    # Remove newline characters from the "volume4wAvg" column
fundamental_df_raw['Open'] = fundamental_df_raw['Open'].str.replace('$', '', regex=False)  # Drop the dollar sign from "Open". was ['$']
fundamental_df_raw['Open'] = fundamental_df_raw['Open'].str.replace('‐', '', regex=False)         # Drop the minus sign from "Open"
fundamental_df_raw['Open'] = fundamental_df_raw['Open'].replace('', None)

fundamental_df_raw['marketCap'] = fundamental_df_raw['marketCap'].str.replace('[$,]', '', regex=True)  # Drop the dollar sign from "Open"
fundamental_df_raw['marketCap'] = fundamental_df_raw['marketCap'].str.replace('‐', '', regex=False)         # Drop the minus sign from "Open"
fundamental_df_raw['marketCap'] = fundamental_df_raw['marketCap'].replace('', None)

fundamental_df_raw['sharesIssued'] = fundamental_df_raw['sharesIssued'].str.replace(',', '')    # Drop the dollar sign from "sharesIssued"
fundamental_df_raw['EPS'] = fundamental_df_raw['EPS'].str.replace('$', '', regex=False)    # Drop the dollar sign from "EPS"
fundamental_df_raw['DPS'] = fundamental_df_raw['DPS'].str.replace('$', '', regex=False)    # Drop the dollar sign from "DPS"
fundamental_df_raw['bookValuePerShare'] = fundamental_df_raw['bookValuePerShare'].str.replace('$', '')    # Drop the dollar sign from "bookValuePerShare"
fundamental_df_raw['Breakdown'] = fundamental_df_raw['Breakdown'].str.replace('\n', ' ')    # Remove newline characters from the "Bid_Ask" column
fundamental_df_raw.head(5)

,lastPrice,Change,Bid_Ask,volumePerDay,volume4wAvg,Open,dayRange,prevClose,lastTrade,oneWeek,...,sharesIssued,Sector,similarCompanies,EPS,DPS,bookValuePerShare,Breakdown,Recommendation,lastUpdated,Ticker
0,31.26,1.02 (3.37%),$29.88 - $31.35,160245,245483,31.07,$30.56 - $31.45,$30.24,6:58pm 03/11 (AEDT),+3.30%,...,82220441,Consumer Cyclical,CTD / WEB / SUL,1.079,0.62,0.60,6 Buy · 5 Hold · 2 Sell,Hold,01/10/23,ARB
1,11.08,0.17 (1.56%),$10.77 - $11.35,366353,441797,10.95,$10.725 - $11.10,$10.91,6:58pm 03/11 (AEDT),+2.59%,...,140894696,Consumer Cyclical,SGR / LNW / SKC,0.693,0.39,0.526,11 Buy · 2 Hold · 0 Sell,Strong Buy,01/10/23,GUD
2,10.12,0.07 (0.70%),$9.96 - $10.92,37398,127350,10.16,$10.06 - $10.22,$10.05,6:58pm 03/11 (AEDT),-0.88%,...,100484131,Consumer Cyclical,BGP / CTT / AX1,0.217,0.125,0.722,7 Buy · 4 Hold · 1 Sell,Buy,01/10/23,PWH
3,16.10,0.65 (-3.88%),$16.15 - $17.10,5570,758,16.75,$16.01 - $16.75,$16.75,6:58pm 03/11 (AEDT),-3.88%,...,13569557,Consumer Cyclical,NDO / ALG / BST,1.811,0.90,3.324,NaN,NaN,NaN,SFC
4,0.89,0.00 (0.00),$0.845 - $0.90,0,11146,None,‐ - ‐,$0.89,6:58pm 03/11 (AEDT),+3.49%,...,81285172,Consumer Cyclical,ISU / KED / CCX,0.084,0.0233,0.986,NaN,NaN,NaN,AHL


In [10]:
# Check dataframe datatypes
print(fundamental_df_raw.dtypes)
print(fundamental_df_raw.columns)

lastPrice            object
Change               object
Bid_Ask              object
volumePerDay         object
volume4wAvg          object
Open                 object
dayRange             object
prevClose            object
lastTrade            object
oneWeek              object
oneMonth             object
YTD2023              object
oneYear              object
vsSectorOneYr        object
vsASX200OneYr        object
marketCap            object
ASXRank              object
sectorRank           object
sharesIssued         object
Sector               object
similarCompanies     object
EPS                  object
DPS                  object
bookValuePerShare    object
Breakdown            object
Recommendation       object
lastUpdated          object
Ticker               object
dtype: object
Index(['lastPrice', 'Change', 'Bid_Ask', 'volumePerDay', 'volume4wAvg', 'Open',
       'dayRange', 'prevClose', 'lastTrade', 'oneWeek', 'oneMonth', 'YTD2023',
       'oneYear', 'vsSectorOneYr', 'vsASX20

In [11]:
# Change data types for  all columns
fundamental_df_raw['lastPrice'] = fundamental_df_raw['lastPrice'].astype(float)                  # Change to float
fundamental_df_raw['Change'] = fundamental_df_raw['Change'].astype(str)                          # Change to string
fundamental_df_raw['Bid_Ask'] = fundamental_df_raw['Bid_Ask'].astype(str)                        # Change to string
fundamental_df_raw['volumePerDay'] = fundamental_df_raw['volumePerDay'].astype(float)              # Change to float
fundamental_df_raw['volume4wAvg'] = fundamental_df_raw['volume4wAvg'].astype(float)                # Change to float
fundamental_df_raw['Open'] = fundamental_df_raw['Open'].astype(float)                            # Change to float
fundamental_df_raw['dayRange'] = fundamental_df_raw['dayRange'].astype(str)                      # Change to string
fundamental_df_raw['prevClose'] = fundamental_df_raw['prevClose'].astype(str)                    # Change to string
fundamental_df_raw['lastTrade'] = fundamental_df_raw['lastTrade'].astype(str)                    # Change to string
fundamental_df_raw['oneWeek'] = fundamental_df_raw['oneWeek'].astype(str)                        # Change to string
fundamental_df_raw['oneMonth'] = fundamental_df_raw['oneMonth'].astype(str)                      # Change to string
fundamental_df_raw['YTD2023'] = fundamental_df_raw['YTD2023'].astype(str)                        # Change to string
fundamental_df_raw['sectorRank'] = fundamental_df_raw['sectorRank'].astype(str)                  # Change to string
fundamental_df_raw['vsSectorOneYr'] = fundamental_df_raw['vsSectorOneYr'].astype(str)            # Change to string
fundamental_df_raw['vsASX200OneYr'] = fundamental_df_raw['vsASX200OneYr'].astype(str)            # Change to string
fundamental_df_raw['marketCap'] = fundamental_df_raw['marketCap'].astype(float)                    # Change to float
fundamental_df_raw['ASXRank'] = fundamental_df_raw['ASXRank'].astype(str)                        # Change to string
fundamental_df_raw['sectorRank'] = fundamental_df_raw['sectorRank'].astype(str)                  # Change to float
fundamental_df_raw['sharesIssued'] = fundamental_df_raw['sharesIssued'].astype(float)              # Change to integer
fundamental_df_raw['Sector'] = fundamental_df_raw['Sector'].astype(str)                          # Change to string
fundamental_df_raw['similarCompanies'] = fundamental_df_raw['similarCompanies'].astype(str)      # Change to float
fundamental_df_raw['EPS'] = fundamental_df_raw['EPS'].astype(float)                              # Change to float
fundamental_df_raw['DPS'] = fundamental_df_raw['DPS'].astype(float)                              # Change to float
fundamental_df_raw['bookValuePerShare'] = fundamental_df_raw['bookValuePerShare'].astype(float)  # Change to float
fundamental_df_raw['Breakdown'] = fundamental_df_raw['Breakdown'].astype(str)                    # Change to string
fundamental_df_raw['Recommendation'] = fundamental_df_raw['Recommendation'].astype(str)          # Change to string
fundamental_df_raw['lastUpdated'] = fundamental_df_raw['lastUpdated'].astype(str)                # Change to string
fundamental_df_raw['Ticker'] = fundamental_df_raw['Ticker'].astype(str)                          # Change to string
# Print the DataFrame with updated data types
print(fundamental_df_raw.head(5))
print(fundamental_df_raw.dtypes)

   lastPrice         Change          Bid_Ask  volumePerDay  volume4wAvg  \
0      31.26   1.02 (3.37%)  $29.88 - $31.35      160245.0     245483.0   
1      11.08   0.17 (1.56%)  $10.77 - $11.35      366353.0     441797.0   
2      10.12   0.07 (0.70%)   $9.96 - $10.92       37398.0     127350.0   
3      16.10  0.65 (-3.88%)  $16.15 - $17.10        5570.0        758.0   
4       0.89    0.00 (0.00)   $0.845 - $0.90           0.0      11146.0   

    Open          dayRange prevClose            lastTrade oneWeek  ...  \
0  31.07   $30.56 - $31.45    $30.24  6:58pm 03/11 (AEDT)  +3.30%  ...   
1  10.95  $10.725 - $11.10    $10.91  6:58pm 03/11 (AEDT)  +2.59%  ...   
2  10.16   $10.06 - $10.22    $10.05  6:58pm 03/11 (AEDT)  -0.88%  ...   
3  16.75   $16.01 - $16.75    $16.75  6:58pm 03/11 (AEDT)  -3.88%  ...   
4    NaN             ‐ - ‐     $0.89  6:58pm 03/11 (AEDT)  +3.49%  ...   

  sharesIssued             Sector similarCompanies    EPS     DPS  \
0   82220441.0  Consumer Cyclical  

In [12]:
# Assuming you have a DataFrame with 'lastPrice' and 'EPS' columns
# You can calculate the P/E ratio and store it in a new column 'PE' as follows:
# Avoid division by zero: set 'PE' to NaN if 'EPS' is zero
fundamental_df_raw['PE'] = round(fundamental_df_raw['lastPrice'] / fundamental_df_raw['EPS'], 3)
fundamental_df_raw['PE'] = fundamental_df_raw['PE'].where(fundamental_df_raw['EPS'] != 0, 0)

print(fundamental_df_raw.dtypes)

lastPrice            float64
Change                object
Bid_Ask               object
volumePerDay         float64
volume4wAvg          float64
Open                 float64
dayRange              object
prevClose             object
lastTrade             object
oneWeek               object
oneMonth              object
YTD2023               object
oneYear               object
vsSectorOneYr         object
vsASX200OneYr         object
marketCap            float64
ASXRank               object
sectorRank            object
sharesIssued         float64
Sector                object
similarCompanies      object
EPS                  float64
DPS                  float64
bookValuePerShare    float64
Breakdown             object
Recommendation        object
lastUpdated           object
Ticker                object
PE                   float64
dtype: object


In [13]:
# Assuming you have a DataFrame named 'df'
fundamental_df_raw['fundamental_ID'] = range(1, len(fundamental_df_raw) + 1)
fundamental_df_raw.head(5)

,lastPrice,Change,Bid_Ask,volumePerDay,volume4wAvg,Open,dayRange,prevClose,lastTrade,oneWeek,...,similarCompanies,EPS,DPS,bookValuePerShare,Breakdown,Recommendation,lastUpdated,Ticker,PE,fundamental_ID
0,31.26,1.02 (3.37%),$29.88 - $31.35,160245.0,245483.0,31.07,$30.56 - $31.45,$30.24,6:58pm 03/11 (AEDT),+3.30%,...,CTD / WEB / SUL,1.079,0.6200,0.600,6 Buy · 5 Hold · 2 Sell,Hold,01/10/23,ARB,28.971,1
1,11.08,0.17 (1.56%),$10.77 - $11.35,366353.0,441797.0,10.95,$10.725 - $11.10,$10.91,6:58pm 03/11 (AEDT),+2.59%,...,SGR / LNW / SKC,0.693,0.3900,0.526,11 Buy · 2 Hold · 0 Sell,Strong Buy,01/10/23,GUD,15.988,2
2,10.12,0.07 (0.70%),$9.96 - $10.92,37398.0,127350.0,10.16,$10.06 - $10.22,$10.05,6:58pm 03/11 (AEDT),-0.88%,...,BGP / CTT / AX1,0.217,0.1250,0.722,7 Buy · 4 Hold · 1 Sell,Buy,01/10/23,PWH,46.636,3
3,16.10,0.65 (-3.88%),$16.15 - $17.10,5570.0,758.0,16.75,$16.01 - $16.75,$16.75,6:58pm 03/11 (AEDT),-3.88%,...,NDO / ALG / BST,1.811,0.9000,3.324,nan,nan,nan,SFC,8.890,4
4,0.89,0.00 (0.00),$0.845 - $0.90,0.0,11146.0,NaN,‐ - ‐,$0.89,6:58pm 03/11 (AEDT),+3.49%,...,ISU / KED / CCX,0.084,0.0233,0.986,nan,nan,nan,AHL,10.595,5


In [14]:
fundamental_df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   lastPrice          259 non-null    float64
 1   Change             259 non-null    object 
 2   Bid_Ask            259 non-null    object 
 3   volumePerDay       259 non-null    float64
 4   volume4wAvg        259 non-null    float64
 5   Open               245 non-null    float64
 6   dayRange           259 non-null    object 
 7   prevClose          259 non-null    object 
 8   lastTrade          259 non-null    object 
 9   oneWeek            259 non-null    object 
 10  oneMonth           259 non-null    object 
 11  YTD2023            259 non-null    object 
 12  oneYear            259 non-null    object 
 13  vsSectorOneYr      259 non-null    object 
 14  vsASX200OneYr      259 non-null    object 
 15  marketCap          259 non-null    float64
 16  ASXRank            259 non

In [15]:
# Merge based on 'columnA' and 'columnB'
merged_df = pd.merge(fundamental_df_raw, df, left_on='Ticker', right_on='ticker', how='inner')

# Display the merged DataFrame
merged_df.head()

,lastPrice,Change,Bid_Ask,volumePerDay,volume4wAvg,Open,dayRange,prevClose,lastTrade,oneWeek,...,Breakdown,Recommendation,lastUpdated,Ticker,PE,fundamental_ID,ticker,company_name,industry_gp,market_cap
0,31.26,1.02 (3.37%),$29.88 - $31.35,160245.0,245483.0,31.07,$30.56 - $31.45,$30.24,6:58pm 03/11 (AEDT),+3.30%,...,6 Buy · 5 Hold · 2 Sell,Hold,01/10/23,ARB,28.971,1,ARB,ARB CORPORATION LIMITED.,Automobiles & Components,2.455925e+09
1,11.08,0.17 (1.56%),$10.77 - $11.35,366353.0,441797.0,10.95,$10.725 - $11.10,$10.91,6:58pm 03/11 (AEDT),+2.59%,...,11 Buy · 2 Hold · 0 Sell,Strong Buy,01/10/23,GUD,15.988,2,GUD,G.U.D. HOLDINGS LIMITED,Automobiles & Components,1.534343e+09
2,10.12,0.07 (0.70%),$9.96 - $10.92,37398.0,127350.0,10.16,$10.06 - $10.22,$10.05,6:58pm 03/11 (AEDT),-0.88%,...,7 Buy · 4 Hold · 1 Sell,Buy,01/10/23,PWH,46.636,3,PWH,PWR HOLDINGS LIMITED,Automobiles & Components,1.000822e+09
3,16.10,0.65 (-3.88%),$16.15 - $17.10,5570.0,758.0,16.75,$16.01 - $16.75,$16.75,6:58pm 03/11 (AEDT),-3.88%,...,nan,nan,nan,SFC,8.890,4,SFC,SCHAFFER CORPORATION LIMITED,Automobiles & Components,2.272901e+08
4,0.89,0.00 (0.00),$0.845 - $0.90,0.0,11146.0,NaN,‐ - ‐,$0.89,6:58pm 03/11 (AEDT),+3.49%,...,nan,nan,nan,AHL,10.595,5,AHL,ADRAD HOLDINGS LIMITED,Automobiles & Components,7.275023e+07


In [17]:
# You can rearrange the columns to have 'Ticker' as the first column
fundamental_df_clean = merged_df[[ 'fundamental_ID', 'Ticker', 'company_name', 'industry_gp', 'lastPrice', 'Change', 'Bid_Ask', 'volumePerDay', 'volume4wAvg', 'Open',
       'dayRange', 'prevClose', 'lastTrade', 'oneWeek', 'oneMonth', 'YTD2023',
       'oneYear', 'vsSectorOneYr', 'vsASX200OneYr', 'marketCap', 'ASXRank',
       'sectorRank', 'sharesIssued', 'Sector', 'similarCompanies', 'EPS',
       'DPS', 'bookValuePerShare', 'Breakdown', 'Recommendation',
       'lastUpdated', 'PE']]
fundamental_df_clean.head(5)

,fundamental_ID,Ticker,company_name,industry_gp,lastPrice,Change,Bid_Ask,volumePerDay,volume4wAvg,Open,...,sharesIssued,Sector,similarCompanies,EPS,DPS,bookValuePerShare,Breakdown,Recommendation,lastUpdated,PE
0,1,ARB,ARB CORPORATION LIMITED.,Automobiles & Components,31.26,1.02 (3.37%),$29.88 - $31.35,160245.0,245483.0,31.07,...,82220441.0,Consumer Cyclical,CTD / WEB / SUL,1.079,0.6200,0.600,6 Buy · 5 Hold · 2 Sell,Hold,01/10/23,28.971
1,2,GUD,G.U.D. HOLDINGS LIMITED,Automobiles & Components,11.08,0.17 (1.56%),$10.77 - $11.35,366353.0,441797.0,10.95,...,140894696.0,Consumer Cyclical,SGR / LNW / SKC,0.693,0.3900,0.526,11 Buy · 2 Hold · 0 Sell,Strong Buy,01/10/23,15.988
2,3,PWH,PWR HOLDINGS LIMITED,Automobiles & Components,10.12,0.07 (0.70%),$9.96 - $10.92,37398.0,127350.0,10.16,...,100484131.0,Consumer Cyclical,BGP / CTT / AX1,0.217,0.1250,0.722,7 Buy · 4 Hold · 1 Sell,Buy,01/10/23,46.636
3,4,SFC,SCHAFFER CORPORATION LIMITED,Automobiles & Components,16.10,0.65 (-3.88%),$16.15 - $17.10,5570.0,758.0,16.75,...,13569557.0,Consumer Cyclical,NDO / ALG / BST,1.811,0.9000,3.324,nan,nan,nan,8.890
4,5,AHL,ADRAD HOLDINGS LIMITED,Automobiles & Components,0.89,0.00 (0.00),$0.845 - $0.90,0.0,11146.0,NaN,...,81285172.0,Consumer Cyclical,ISU / KED / CCX,0.084,0.0233,0.986,nan,nan,nan,10.595


In [18]:
fundamental_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   fundamental_ID     259 non-null    int64  
 1   Ticker             259 non-null    object 
 2   company_name       259 non-null    object 
 3   industry_gp        259 non-null    object 
 4   lastPrice          259 non-null    float64
 5   Change             259 non-null    object 
 6   Bid_Ask            259 non-null    object 
 7   volumePerDay       259 non-null    float64
 8   volume4wAvg        259 non-null    float64
 9   Open               245 non-null    float64
 10  dayRange           259 non-null    object 
 11  prevClose          259 non-null    object 
 12  lastTrade          259 non-null    object 
 13  oneWeek            259 non-null    object 
 14  oneMonth           259 non-null    object 
 15  YTD2023            259 non-null    object 
 16  oneYear            259 non

In [19]:
# Export the DataFrame to a CSV file
fundamental_df_clean.to_csv('Resources/07_ASX_Fundamental_Final_Clean.csv', index=False)

In [20]:
# Use the read_csv function to read the CSV file again into
fundamental_df_clean = pd.read_csv("Resources/07_ASX_Fundamental_Final_Clean.csv")

fundamental_df_clean.head(5)

print(fundamental_df_clean.dtypes)

fundamental_ID         int64
Ticker                object
company_name          object
industry_gp           object
lastPrice            float64
Change                object
Bid_Ask               object
volumePerDay         float64
volume4wAvg          float64
Open                 float64
dayRange              object
prevClose             object
lastTrade             object
oneWeek               object
oneMonth              object
YTD2023               object
oneYear               object
vsSectorOneYr         object
vsASX200OneYr         object
marketCap            float64
ASXRank               object
sectorRank            object
sharesIssued         float64
Sector                object
similarCompanies      object
EPS                  float64
DPS                  float64
bookValuePerShare    float64
Breakdown             object
Recommendation        object
lastUpdated           object
PE                   float64
dtype: object
